# Convolutional Neural Networks for Visual Recognition
## Image Classification
**Motivation-** In this section we will introduce the Image Classification problem, which is the task of assigning an input image one label from a fixed set of categories. This is one of the core problems in Computer Vision that, despite its simplicity, has a large variety of practical applications.

**Example-** For example, in the image below an image classification model takes a single image and assigns probabilities to 4 labels, {cat, dog, hat, mug}. As shown in the image, keep in mind that to a computer an image is represented as one large 3-dimensional array of numbers. In this example, the cat image is 248 pixels wide, 400 pixels tall, and has three color channels Red,Green,Blue (or RGB for short). Therefore, the image consists of 248 x 400 x 3 numbers, or a total of 297,600 numbers. Each number is an integer that ranges from 0 (black) to 255 (white). Our task is to turn this quarter of a million numbers into a single label, such as “cat”.
![Classification Problem](Images/visual_data.png)

**Challenges-** Since this task of recognizing a visual concept (e.g. cat) is relatively trivial for a human to perform, it is worth considering the challenges involved from the perspective of a Computer Vision algorithm. As we present (an inexhaustive) list of challenges below, keep in mind the raw representation of Images as a 3-D array of brightness values:

* **Viewpoint variation-** A single instance of an object can be oriented in many ways with respect to the camera.
* **Scale variation-** Visual classes often exhibit variation in their size (size in the real world, not only in terms of their extent in the image).
* **Deformation-** Many objects of interest are not rigid bodies and can be deformed in extreme ways.
* **Occlusion-** The objects of interest can be occluded. Sometimes only a small portion of an object (as little as few pixels) could be visible.
* **Illumination conditions-** The effects of illumination are drastic on the pixel level.
* **Background clutter-** The objects of interest may blend into their environment, making them hard to identify.
* **Intra-class variation-** The classes of interest can often be relatively broad, such as chair. There are many different types of these objects, each with their own appearance.
A good image classification model must be invariant to the cross product of all these variations, while simultaneously retaining sensitivity to the inter-class variations. The below image explains all these challenges very efficiently and elegantly-
![Images Classification Challenges](Images/challenges.jpeg)

**Data-driven approach-** How might we go about writing an algorithm that can classify Images into distinct categories? Unlike writing an algorithm for, for example, sorting a list of numbers, it is not obvious how one might write an algorithm for identifying cats in Images. Therefore, instead of trying to specify what every one of the categories of interest look like directly in code, the approach that we will take is not unlike one you would take with a child: we’re going to provide the computer with many examples of each class and then develop learning algorithms that look at these examples and learn about the visual appearance of each class. This approach is referred to as a data-driven approach, since it relies on first accumulating a training dataset of labeled Images. Here is an example of what such a dataset might look like:

![Sample Training Sets](Images/cifar10.jpg)
**Left:** Example Images from the CIFAR-10 dataset. **Right:** first column shows a few test Images and next to each we show the top 10 nearest neighbors in the training set according to pixel-wise difference.

**The image classification pipeline:** We’ve seen that the task in Image Classification is to take an array of pixels that represents a single image and assign a label to it. Our complete pipeline can be formalized as follows:

* **Input:** Our input consists of a set of N Images, each labeled with one of K different classes. We refer to this data as the training set.
* **Learning:** Our task is to use the training set to learn what every one of the classes looks like. We refer to this step as training a classifier, or learning a model.
* **Evaluation:** In the end, we evaluate the quality of the classifier by asking it to predict labels for a new set of Images that it has never seen before. We will then compare the true labels of these Images to the ones predicted by the classifier. Intuitively, we’re hoping that a lot of the predictions match up with the true answers (which we call the ground truth).

### Nearest Neighbor Classifier
This classifier has nothing to do with Convolutional Neural Networks and it is very rarely used in practice, but it will allow us to get an idea about the basic approach to an image classification problem.

**Example image classification dataset: CIFAR-10 - ** One popular toy image classification dataset is the CIFAR-10 dataset. This dataset consists of 60,000 tiny Images that are 32 pixels high and wide. Each image is labeled with one of 10 classes (for example “airplane, automobile, bird, etc”). These 60,000 Images are partitioned into a training set of 50,000 Images and a test set of 10,000 Images. In the image below you can see 10 random example Images from each one of the 10 classes:

Suppose now that we are given the CIFAR-10 training set of 50,000 Images (5,000 Images for every one of the labels), and we wish to label the remaining 10,000. The nearest neighbor classifier will take a test image, compare it to every single one of the training Images, and predict the label of the closest training image. In the image above and on the right you can see an example result of such a procedure for 10 example test Images. Notice that in only about 3 out of 10 examples an image of the same class is retrieved, while in the other 7 examples this is not the case. For example, in the 8th row the nearest training image to the horse head is a red car, presumably due to the strong black background. As a result, this image of a horse would in this case be mislabeled as a car.

You may have noticed that we left unspecified the details of exactly how we compare two Images, which in this case are just two blocks of 32 x 32 x 3. One of the simplest possibilities is to compare the Images pixel by pixel and add up all the differences. In other words, given two Images and representing them as vectors $I_1$,$I_2$ , a reasonable choice for comparing them might be the L1 distance:

$$d_1(I_1,I_2)=\displaystyle\sum_{p}|I_1^p−I_2^p|$$

Where the sum is taken over all pixels. Here is the procedure visualized:

![Error Calculation](Images/nearestNeighbourErrorMetric.jpeg)

Let us implement a simple nearest neighbour classifier using the above measure of proximity:

In [ ]:
import numpy as np
import pickle
import matplotlib.pyplot as plt
import cv2

In [ ]:
def loadData(file_name):
    file_pointer = open(file_name,'rb')
    file_data = pickle.load(file_pointer)
    feature_vector = file_data['data']
    label = np.array(file_data['labels'])
    return (feature_vector,label)

In [ ]:
import numpy as np

class NearestNeighbour(object):
    def __init__(self):
        self.PX = np.array([])
        self.Py = np.array([])
    def train(self,X,y):
        self.PX = X
        self.Py = y
    def predict(self,X):
        sh = X.shape[0]
        prediction = np.zeros((sh))
        for i in range(sh):
            #proximity = np.sum(np.abs(self.PX - X[i,:]),axis=1)
            proximity = np.sqrt(np.sum(np.square(self.PX-X[i,:]),axis=1))
            min_index = np.argmin(proximity)
            prediction[i] = self.Py[min_index]
        return prediction

In [ ]:
if __name__ == '__main__':
    file_name = 'Datasets\\data_batch_1'
    feature_vector, label = loadData(file_name)
    NNeighbor = NearestNeighbour()
    NNeighbor.train(feature_vector[:7000,:],label[:7000])
    output = NNeighbor.predict(feature_vector[7000:,:])
    label_test = label[7000:]
    accuracy = np.mean(label_test == output)
    print accuracy

If you ran this code, you would see that this classifier only achieves **38.6%** on CIFAR-10(and 31.01% for 10000 traning Images). That’s more impressive than guessing at random (which would give 10% accuracy since there are 10 classes), but nowhere near **human performance** (which is estimated at about **94%**) or near state-of-the-art **Convolutional Neural Networks** that achieve about **95%**, matching human accuracy.

**The choice of distance:** There are many other ways of computing distances between vectors. Another common choice could be to instead use the L2 distance, which has the geometric interpretation of computing the euclidean distance between two vectors. The distance takes the form:

$$ d_2 (I_1, I_2) = \sqrt{\sum_{p} \left( I^p_1 - I^p_2 \right)^2} $$

**L1 vs. L2:** It is interesting to consider differences between the two metrics. In particular, the L2 distance is much more unforgiving than the L1 distance when it comes to differences between two vectors. That is, the L2 distance prefers many medium disagreements to one big one. L1 and L2 distances (or equivalently the L1/L2 norms of the differences between a pair of Images) are the most commonly used special cases of a p-norm. If you ran the Nearest Neighbor classifier on CIFAR-10 with this distance, you would obtain **35.4%** accuracy (slightly lower than our L1 distance result).


### k - Nearest Neighbor Classifier

You may have noticed that it is strange to only use the label of the nearest image when we wish to make a prediction. Indeed, it is almost always the case that one can do better by using what’s called a k-Nearest Neighbor Classifier. The idea is very simple: instead of finding the single closest image in the training set, we will find the top k closest Images, and have them vote on the label of the test image. In particular, when k = 1, we recover the Nearest Neighbor classifier. Intuitively, higher values of k have a smoothing effect that makes the classifier more resistant to outliers:
![K-Nearest Neighbour Classifier](Images/knn.jpeg)

In practice, you will almost always want to use k-Nearest Neighbor. But what value of k should you use?

### Validation sets for Hyperparameter tuning
The k-nearest neighbor classifier requires a setting for k. But what number works best? Additionally, we saw that there are many different distance functions we could have used: L1 norm, L2 norm, there are many other choices we didn’t even consider (e.g. dot products). These choices are called **hyperparameters** and they come up very often in the design of many Machine Learning algorithms that learn from data. It’s often not obvious what values/settings one should choose.

You might be tempted to suggest that we should try out many different values and see what works best. That is a fine idea and that’s indeed what we will do, but this must be done very carefully. In particular, we cannot use the test set for the purpose of tweaking hyperparameters.Whenever you’re designing Machine Learning algorithms, you should think of the test set as a very precious resource that should ideally never be touched until one time at the very end. Otherwise, the very real danger is that you may tune your hyperparameters to work well on the test set, but if you were to deploy your model you could see a significantly reduced performance. In practice, we would say that you **overfit** to the test set. Another way of looking at it is that if you tune your hyperparameters on the test set, you are effectively using the test set as the training set, and therefore the performance you achieve on it will be too optimistic with respect to what you might actually observe when you deploy your model. But if you only use the test set once at end, it remains a good proxy for measuring the **generalization** of your classifier.

![Overfitting Problem](Images/Overfitting.png)

**Cross-validation-** In cases where the size of your training data (and therefore also the validation data) might be small, people sometimes use a more sophisticated technique for hyperparameter tuning called cross-validation. Working with our previous example, the idea is that instead of arbitrarily picking the first 1000 datapoints to be the validation set and rest training set, you can get a better and less noisy estimate of how well a certain value of k works by iterating over different validation sets and averaging the performance across these. For example, in 5-fold cross-validation, we would split the training data into 5 equal folds, use 4 of them for training, and 1 for validation. We would then iterate over which fold is the validation fold, evaluate the performance, and finally average the performance across the different folds.
![Cross Validation](Images\crossval.jpeg)

> Evaluate on the test set only a single time, at the very end.

If your data is very high-dimensional, consider using a dimensionality reduction technique such as **PCA(Pricipal Component Analysis)**, let's look, how dimentionality reduction is done using Scikit Learn module: 

In [ ]:
from sklearn.decomposition import PCA

file_name = 'Datasets\\data_batch_1'
feature_vector, label = loadData(file_name)
print('Size of data before transformation: {}'.format(feature_vector.shape))
pca = PCA(n_components=300)
pca.fit(feature_vector)
transformed_feature_vector = pca.transform(feature_vector)
print('Size of data after transformation: {}'.format(transformed_feature_vector.shape))

In [ ]:
if __name__ == '__main__':
    NNeighbor = NearestNeighbour()
    NNeighbor.train(transformed_feature_vector[:7000,:],label[:7000])
    output = NNeighbor.predict(transformed_feature_vector[7000:,:])
    label_test = label[7000:]
    accuracy = np.mean(label_test == output)
    print accuracy

You can clearly see that PCA helped in increasing accuracy as well as speed up the whole by reducing dimentions of the feature vector by a factor of ten. PCA eliminates all the dimentions that does not contribute to variance, in simple terms, even if some dimentions is eliminted from data, it's not going to have a great deal of impact on the data.    

### Pros and Cons of Nearest Neighbor classifier
#### Pros-
* It is very simple to implement and understand
* The classifier takes no time to train, , since all that is required is to store and possibly index the training data.
* Effective for smaller datasets with less number of dimentions.
#### Cons-
* We are paying huge computational cost at test time, since classifying a test example requires a comparison to every single training example.
* The classifier must remember all of the training data and store it for future comparisons with the test data.

## Applying kNN in practice

If you wish to apply kNN in practice (hopefully not on Images) proceed as follows:

1. **Preprocess your data:** Normalize the features in your data (e.g. one pixel in Images) to have zero mean and unit variance.
2. **Dimensionality reduction:**If your data is very high-dimensional, consider using a dimensionality reduction technique such as PCA(Principle Component Analysis).
3. **Cross Validation:**Split your training data randomly into train/val splits. As a rule of thumb, between 70-90% of your data usually goes to the train split. This setting depends on how many hyperparameters you have and how much of an influence you expect them to have.
4. **Use libraries to accelerate the process:**If your kNN classifier is running too long, consider using an Approximate Nearest Neighbor library.
5. **Save Hyperparameters:**Take note of the hyperparameters that gave the best results.

## Parameterized mapping from Images to label scores
We are now going to develop a more powerful approach to image classification that we will eventually naturally extend to entire Neural Networks and Convolutional Neural Networks. The approach will have two major components: a score function that maps the raw data to class scores, and a loss function that quantifies the agreement between the predicted scores and the ground truth labels. We will then cast this as an optimization problem in which we will minimize the loss function with respect to the parameters of the score function.
The first component of this approach is to define the score function that maps the pixel values of an image to confidence scores for each class.

### Linear Classification
We are now going to develop a more powerful approach to image classification that we will eventually naturally extend to entire Neural Networks and Convolutional Neural Networks. The approach will have two major components: a **score function** that maps the raw data to class scores, and a **loss function** that quantifies the agreement between the predicted scores and the ground truth labels.

**Linear classifier** In this module we will start out with arguably the simplest possible function, a linear mapping:

$$f(x_i,W,b)=Wx_i+b$$

In the above equation, we are assuming that the image $x_i$ has all of its pixels flattened out to a single column vector of shape $[D \times 1]$. The matrix **W** (of size $[K \times D]$), and the vector **b** (of size $[K \times 1]$) are the **parameters** of the function. In CIFAR-10, $x_i$ contains all pixels in the i-th image flattened into a single $[3072 \times 1]$ column, **W** is $[10 \times 3072]$ and **b** is $[10 \times 1]$, so $3072$ numbers come into the function (the raw pixel values) and 10 numbers come out (the class scores). The parameters in **W** are often called the **weights**, and **b** is called the **bias vector** because it influences the output scores, but without interacting with the actual data $x_i$. However, you will often hear people use the terms _weights_ and _parameters_ interchangeably.

> There are few things to note here:
* First, note that the single matrix multiplication $Wx_i$ is effectively evaluating 10 separate classifiers in parallel (one for each class), where each classifier is a row of W.
* Notice also that we think of the input data $(x_i,y_i)$ as given and fixed, but we have control over the setting of the parameters W,b. Our goal will be to set these in such way that the computed scores match the ground truth labels across the whole training set. We will go into much more detail about how this is done, but intuitively we wish that the correct class has a score that is higher than the scores of incorrect classes.
* An advantage of this approach is that the training data is used to learn the parameters W,b, but once the learning is complete we can discard the entire training set and only keep the learned parameters. That is because a new test image can be simply forwarded through the function and classified based on the computed scores.
* Lastly, note that to classifying the test image involves a single matrix multiplication and addition, which is significantly faster than comparing a test image to all training Images.


### Interpreting a linear classifier

Notice that a linear classifier computes the score of a class as a **weighted sum** of all of its pixel values across all 3 of its color channels. Depending on precisely what values we set for these weights, the function has the capacity to like or dislike (depending on the sign of each weight) certain colors at certain positions in the image. For instance, you can imagine that the “ship” class might be more likely if there is a lot of blue on the sides of an image (which could likely correspond to water). You might expect that the “ship” classifier would then have a lot of positive weights across its blue channel weights (presence of blue increases score of ship), and negative weights in the red/green channels (presence of red/green decreases the score of ship).

![Image Map](Images/scorefunction.jpg)

An example of mapping an image to class scores. For the sake of visualization, we assume the image only has 4 pixels (4 monochrome pixels, we are not considering color channels in this example for brevity), and that we have 3 classes (red (cat), green (dog), blue (ship) class). (Clarification: in particular, the colors here simply indicate 3 classes and are not related to the RGB channels.) We stretch the image pixels into a column and perform matrix multiplication to get the scores for each class. Note that this particular set of weights W is not good at all: the weights assign our cat image a very low cat score. In particular, this set of weights seems convinced that it's looking at a dog.

**Analogy of Images as high-dimensional points:** Since the Images are stretched into high-dimensional column vectors, we can interpret each image as a single point in this space (e.g. each image in CIFAR-10 is a point in 3072-dimensional space of 32x32x3 pixels). Analogously, the entire dataset is a (labeled) set of points.

Since we defined the score of each class as a weighted sum of all image pixels, each class score is a linear function over this space. We cannot visualize 3072-dimensional spaces, but if we imagine squashing all those dimensions into only two dimensions, then we can try to visualize what the classifier might be doing:

![Pixel Space](Images/pixelspace.jpeg)

As we saw above, every row of **$W$** is a classifier for one of the classes. The geometric interpretation of these numbers is that as we change one of the rows of $W$, the corresponding line in the pixel space will rotate in different directions. The biases **$b$**, on the other hand, allow our classifiers to translate the lines. In particular, note that without the bias terms, plugging in $x_i=0$ would always give score of zero regardless of the weights, so all lines would be forced to cross the origin.

**Interpretation of linear classifiers as template matching:** Another interpretation for the weights ** $W$ ** is that each row of $W$ corresponds to a template (or sometimes also called a prototype) for one of the classes. The score of each class for an image is then obtained by comparing each template with the image using an inner product (or dot product) one by one to find the one that “fits” best. With this terminology, the linear classifier is doing template matching, where the templates are learned. Another way to think of it is that we are still effectively doing Nearest Neighbor, but instead of having thousands of training Images we are only using a single image per class (although we will learn it, and it does not necessarily have to be one of the Images in the training set), and we use the (negative) inner product as the distance instead of the L1 or L2 distance.

![Image Template](Images/templates.jpg)

Example learned weights at the end of learning for CIFAR-10. Note that, for example, the ship template contains a lot of blue pixels as expected. This template will therefore give a high score once it is matched against Images of ships on the ocean with an inner product. Additionally, note that the horse template seems to contain a two-headed horse, which is due to both left and right facing horses in the dataset. The linear classifier merges these two modes of horses in the data into a single template. Similarly, the car classifier seems to have merged several modes into a single template which has to identify cars from all sides, and of all colors. In particular, this template ended up being red, which hints that there are more red cars in the CIFAR-10 dataset than of any other color.

**Bias trick** Before moving on we want to mention a common simplifying trick to representing the two parameters $W,b$ as one. Recall that we defined the score function as:

$$f(x_i,W,b)=Wx_i+b$$

As we proceed through the material it is a little cumbersome to keep track of two sets of parameters (the biases $b$ and weights WW) separately. A commonly used trick is to combine the two sets of parameters into a single matrix that holds both of them by extending the vector $x_i$ with one additional dimension that always holds the constant **1** - a default bias dimension. With the extra dimension, the new score function will simplify to a single matrix multiply:

$$f(x_i,W)=Wx_i$$

With our CIFAR-10 example, $x_i$ is now [3073 x 1] instead of [3072 x 1] - (with the extra dimension holding the constant 1), and **W** is now [10 x 3073] instead of [10 x 3072]. The extra column that $W$ now corresponds to the bias **b**. An illustration might help clarify:

![Bais Trick](Images\biasTrick.jpeg)



In [ ]:
def Bais_trick(data,W):
    bais = np.zeros((W.shape[0],1)) + 1
    W = np.append(W,bais,axis=1)
    data_bais = np.tile(bais,(data.shape[0]/bias.shape[0],1))
    data = np.append(data,data_bais,axis=1)
    return(data,W)

In [ ]:
def Test_Weight(data,label,W):
    score = W.dot(data.T)
    prediction = np.argmax(score,axis=0)
    return (np.mean(np.equal(label,prediction)))

### Loss function
Going back to the example image of a cat and its scores for the classes “cat”, “dog” and “ship”, we saw that the particular set of weights in that example was not very good at all: We fed in the pixels that depict a cat but the cat score came out very low (-96.8) compared to the other classes (dog score 437.9 and ship score 61.95). We are going to measure our unhappiness with outcomes such as this one with a loss function (or sometimes also referred to as the cost function or the objective). Intuitively, the loss will be high if we’re doing a poor job of classifying the training data, and it will be low if we’re doing well.

#### Multiclass Support Vector Machine loss
There are several ways to define the details of the loss function. As a first example we will first develop a commonly used loss called the **Multiclass Support Vector Machine (SVM)** loss. The SVM loss is set up so that the SVM “wants” the correct class for each image to a have a score higher than the incorrect classes by some fixed margin $Δ$. Notice that it’s sometimes helpful to anthropomorphise the loss functions as we did above: The SVM “wants” a certain outcome in the sense that the outcome would yield a lower loss (which is good).

Let’s now get more precise. Recall that for the i-th example we are given the pixels of image x_i and the label y_i that specifies the index of the correct class. The score function takes the pixels and computes the vector $f(xi,W)$ of class scores, which we will abbreviate to ss (short for scores). For example, the score for the j-th class is the j-th element: $sj=f(xi,W)j$. The Multiclass SVM loss for the i-th example is then formalized as follows:

$$L_i=\displaystyle\sum_{j_i≠y_i}max(0,s_j−{s_y}_i+Δ)$$

**Example** Lets unpack this with an example to see how it works. Suppose that we have three classes that receive the scores **s=[13,−7,11]**, and that the first class is the true class (i.e. **$ y_i=0 $**). Also assume that **Δ** (a hyperparameter we will go into more detail about soon) is 10. The expression above sums over all incorrect classes (**$ j≠y_i $**), so we get two terms:

> **$$L_i=max(0,−7−13+10)+max(0,11−13+10)$$**

You can see that the first term gives zero since [-7 - 13 + 10] gives a negative number, which is then thresholded to zero with the **max(0,−)** function. We get zero loss for this pair because the correct class score (13) was greater than the incorrect class score (-7) by at least the margin 10. In fact the difference was 20, which is much greater than 10 but the SVM only cares that the difference is at least 10; Any additional difference above the margin is clamped at zero with the max operation. The second term computes [11 - 13 + 10] which gives 8. That is, even though the correct class had a higher score than the incorrect class (13 > 11), it was not greater by the desired margin of 10. The difference was only 2, which is why the loss comes out to 8 (i.e. how much higher the difference would have to be to meet the margin). In summary, the SVM loss function wants the score of the correct class yiyi to be larger than the incorrect class scores by at least by **Δ** (delta). If this is not the case, we will accumulate loss.

Note that in this particular module we are working with linear score functions ( $f(x_i;W)={W_x}_i$ ), so we can also rewrite the loss function in this equivalent form:

$$L_i=\displaystyle\sum_{j_i≠y_i}max(0,w_j^Tx_i−{w_y^T}_ix_i+Δ)$$

where **$w_j$** is the j-th row of **W** reshaped as a column. However, this will not necessarily be the case once we start to consider more complex forms of the score function **f**.

In [ ]:
def L1(X,y,W):
    delta = 1.0                                                                         # Defined delta value
    y = y.reshape(labels.shape[0],1)
    score = W.dot(X.T)                                                                  # Calculating Score 
    correct_class_score = score[np.array(y[:,0]),np.array(np.arange(X.T.shape[1]))]     # Getting correct class score
    margins = np.maximum(0,(score - correct_class_score + delta))                       # Calculating loss 
    margins[[y[:,0]],[np.arange(X.T.shape[1])]] = 0                                     # Setting the loss to 0 for correct class
    loss = np.sum(margins,axis=0)
    return loss

In [ ]:
NO_OF_CLASS = 10

file_name ='Datasets\\data_batch_2'
data, labels = loadData(file_name)                              # Loading data
weight = np.random.rand(10,3072) * 0.001                        # Generating random weights
data, weight = Bais_trick(data,weight)                          
loss = L1(data,labels,weight)
print(loss.shape)

**Regularization** There is one bug with the loss function we presented above. Suppose that we have a dataset and a set of parameters $W$ that correctly classify every example (i.e. all scores are so that all the margins are met, and $L_i=0$ for all $i$). The issue is that this set of $W$ is not necessarily unique: there might be many similar $W$ that correctly classify the examples. One easy way to see this is that if some parameters W correctly classify all examples (so loss is zero for each example), then any multiple of these parameters $λW$ where $λ>1$ will also give zero loss because this transformation uniformly stretches all score magnitudes and hence also their absolute differences. For example, if the difference in scores between a correct class and a nearest incorrect class was 15, then multiplying all elements of W by 2 would make the new difference 30.

In other words, we wish to encode some preference for a certain set of weights $W$ over others to remove this ambiguity. We can do so by extending the loss function with a regularization penalty $R(W)$. The most common regularization penalty is the $L2$ norm that discourages large weights through an elementwise quadratic penalty over all parameters:

$$R(W)=\displaystyle\sum_{k}\displaystyle\sum_{l}{W^2_{k,l}}$$

In the expression above, we are summing up all the squared elements of $W$. Notice that the regularization function is not a function of the data, it is only based on the weights. Including the regularization penalty completes the full Multiclass Support Vector Machine loss, which is made up of two components: the data loss (which is the average loss $L_i$ over all examples) and the regularization loss. That is, the full Multiclass SVM loss becomes:

$$ L =  \underbrace{ \frac{1}{N} \sum_i L_i }_\text{data loss} + \underbrace{ \lambda R(W) }_\text{regularization loss} \\\\ $$

The most appealing property of regularization is that, penalizing large weights tends to improve generalization, because it means that no input dimension can have a very large influence on the scores all by itself. For example, suppose that we have some input vector $x=[1,1,1,1]$ and two weight vectors $w1=[1,0,0,0]$, $w2=[0.25,0.25,0.25,0.25]$. Then $w^T_1x=w^T_2x=1$ so both weight vectors lead to the same dot product, but the L2 penalty of $w1$ is 1.0 while the L2 penalty of $w2$ is only 0.25. Therefore, according to the L2 penalty the weight vector $w2$ would be preferred since it achieves a lower regularization loss. Intuitively, this is because the weights in $w2$ are smaller and more diffuse. Since the L2 penalty prefers smaller and more diffuse weight vectors, the final classifier is encouraged to take into account all input dimensions to small amounts rather than a few input dimensions and very strongly. As we will see later in the class, this effect can improve the generalization performance of the classifiers on test Images and lead to less **_overfitting_**.

### Softmax classifier
Unlike the SVM which treats the outputs $f(x_i,W)$ as (uncalibrated and possibly difficult to interpret) scores for each class, the Softmax classifier gives a slightly more intuitive output (normalized class probabilities) and also has a probabilistic interpretation that we will describe shortly. In the Softmax classifier, the function mapping $f(x_i;W)=Wx_i$ stays unchanged, but we now interpret these scores as the unnormalized log probabilities for each class and replace the hinge loss with a cross-entropy loss that has the form:

$$ L_i = -log\frac{e^{f_{x_i}}}{\displaystyle\sum_{j}e^{j_i}} $$

where we are using the notation $f_j$ to mean the j-th element of the vector of class scores $f$. As before, the full loss for the dataset is the mean of $L_i$ over all training examples together with a regularization term $R(W)$. The following function is called the softmax function:

$$f_j(z)=\frac{e^{z_j}}{\displaystyle\sum_{k}e^{z_k}}$$

**Numeric stability:** When you’re writing code for computing the Softmax function in practice, the intermediate terms $e^{f_{y_i}}$ and $\displaystyle\sum_{j}e^{f_j}$ may be very large due to the exponentials. Dividing large numbers can be numerically unstable, so it is important to use a normalization trick. Notice that if we multiply the top and bottom of the fraction by a constant $C$ and push it into the sum, we get the following (mathematically equivalent) expression:

$$\frac{e^{f_{y_i}}}{\sum_{j}e^{f_j}}=\frac{Ce^{f_{y_i}}}{C\displaystyle\sum_{j}e^{f_j}}=\frac{e^{f_{y_i}+logC}}{\displaystyle\sum_{j}e^{f_j+logC}}$$

We are free to choose the value of $C$. This will not change any of the results, but we can use this value to improve the numerical stability of the computation. A common choice for $C$ is to set $ logC =−max_jf_j $. This simply states that we should shift the values inside the vector ff so that the highest value is zero. In code:

In [ ]:
# Original Score: [123, 456, 789]
# Shift the values of f so that the highest number is 0:
f -= np.max(f)                      # f becomes [-666, -333, 0]
p = np.exp(f) / np.sum(np.exp(f))

### SVM vs. Softmax

![SVM vs. Softmax](Images/svmvssoftmax.png)

Example of the difference between the SVM and Softmax classifiers for one datapoint. In both cases we compute the same score vector f (e.g. by matrix multiplication in this section). The difference is in the interpretation of the scores in f: The SVM interprets these as class scores and its loss function encourages the correct class (class 2, in blue) to have a score higher by a margin than the other class scores. The Softmax classifier instead interprets the scores as (unnormalized) log probabilities for each class and then encourages the (normalized) log probability of the correct class to be high (equivalently the negative of it to be low). The final loss for this example is 1.58 for the SVM and 1.04 (note this is 1.04 using the natural logarithm, not base 2 or base 10) for the Softmax classifier, but note that these numbers are not comparable; They are only meaningful in relation to loss computed within the same classifier and with the same data.

### Visualizing the loss function

The loss functions we’ll look at in this class are usually defined over very high-dimensional spaces (e.g. in CIFAR-10 a linear classifier weight matrix is of size [10 x 3073] for a total of 30,730 parameters), making them difficult to visualize. However, we can still gain some intuitions about one by slicing through the high-dimensional space along rays (1 dimension), or along planes (2 dimensions). For example, we can generate a random weight matrix $W$ (which corresponds to a single point in the space), then march along a ray and record the loss function value along the way. That is, we can generate a random direction W1W1 and compute the loss along this direction by evaluating $L(W+aW1)$ for different values of aa. This process generates a simple plot with the value of aa as the x-axis and the value of the loss function as the y-axis. We can also carry out the same procedure with two dimensions by evaluating the loss $L(W+aW1+bW2)$ as we vary $a,b$. In a plot, $a,b$ could then correspond to the x-axis and the y-axis, and the value of the loss function can be visualized with a color:

| One-Dimensional Loss Slice | Two-Dimensional Loss Slice| Two-Dimensional Loss Slice |
| :---: | :---: | :---: |
| ![Loss Function Visualized](Images/svm1d.png)  | ![Loss Function Visualized](Images/svm_one.jpg)  |![Loss Function Visualized](Images/svm_all.jpg)|

We can explain the piecewise-linear structure of the loss function by examining the math. For a single example we have:

$$ L_i = \sum_{j\neq y_i} \left[ \max(0, w_j^Tx_i - w_{y_i}^Tx_i + 1) \right] $$

It is clear from the equation that the data loss for each example is a sum of (zero-thresholded due to the $max(0,−)$ function) linear functions of $W$. Moreover, each row of $W$ (i.e. $w_j$) sometimes has a positive sign in front of it (when it corresponds to a wrong class for an example), and sometimes a negative sign (when it corresponds to the correct class for that example). To make this more explicit, consider a simple dataset that contains three 1-dimensional points and three classes. The full SVM loss (without regularization) becomes:



$$ L_0 = \max(0, w_1^Tx_0 - w_0^Tx_0 + 1) + \max(0, w_2^Tx_0 - w_0^Tx_0 + 1) $$
$$ L_1 = \max(0, w_0^Tx_1 - w_1^Tx_1 + 1) + \max(0, w_2^Tx_1 - w_1^Tx_1 + 1) $$
$$ L_2 = \max(0, w_0^Tx_2 - w_2^Tx_2 + 1) + \max(0, w_1^Tx_2 - w_2^Tx_2 + 1) $$
$$ L = (L_0 + L_1 + L_2)/3 $$

Since these examples are 1-dimensional, the data $x_i$ and weights $w_j$ are numbers. Looking at, for instance, $w_0$, some terms above are linear functions of w0w0 and each is clamped at zero. We can visualize this as follows:

![Sum of Individual Linear Classifier](Images/svmbowl.png)
1-dimensional illustration of the data loss. The x-axis is a single weight and the y-axis is the loss. The data loss is a sum of multiple terms, each of which is either independent of a particular weight, or a linear function of it that is thresholded at zero. The full SVM data loss is a 30,730-dimensional version of this shape.

You may have guessed from its bowl-shaped appearance that the SVM cost function is an example of a convex function. Once we extend our score functions ff to Neural Networks our objective functions will become non-convex, and the visualizations above will not feature bowls but complex, bumpy terrains.

![Convex Functions](Images/convex.jpg)

## Optimization
**Optimization**: It is the process of finding the set of parameters $W$ that minimize the loss function. Now there are very different types of solution to this problem. Lets look at some of them-  

**Strategy #1: A first very bad idea solution: Random search**  
Since it is so simple to check how good a given set of parameters W is, the first (very bad) idea that may come to mind is to simply try out many different random weights and keep track of what works best. 

In [ ]:
file_name ='Datasets\\data_batch_2'
X_train, Y_train = loadData(file_name)

bestloss = float("inf")                                         # Python assigns the highest possible float value
for num in xrange(300):
  W = np.random.randn(10, 3072) * 0.0001                        # generate random parameters
  X_train_transformed = X_train
  X_train_transformed, W = Bais_trick(X_train,W)
  loss = np.mean(L1(X_train_transformed, Y_train, W))           # get the loss over the entire training set
  if loss < bestloss:                                           # keep track of the best solution
    bestloss = loss
    bestW = W
print('Accuracy : {}'.format(Test_Weight(X_train_transformed,Y_train,bestW)))

**Strategy #2: Random Local Search**

**Blindfolded hiker analogy:** One analogy that you may find helpful going forward is to think of yourself as hiking on a hilly terrain with a blindfold on, and trying to reach the bottom. In the example of CIFAR-10, the hills are 30,730-dimensional, since the dimensions of W are 10 x 3073. At every point on the hill we achieve a particular loss (the height of the terrain).

The first strategy you may think of is to try to extend one foot in a random direction and then take a step only if it leads downhill. Concretely, we will start out with a random $W$, generate random perturbations $δW$ to it and if the loss at the perturbed $W+δW$ is lower, we will perform an update. The code for this procedure is as follows:

In [ ]:
import numpy as np
import pickle

data = np.array([])
labels = np.array([])

def getData(file):
    with open(file,'rb') as fo:
        dict = pickle.load(fo, encoding='latin1')
        return (dict['data'], np.array(dict['labels']))

def load_CIFAR10(file):
    flag = 1;
    if type(file) is list:
        for fp in file:
            dat, lab = getData(fp)
            if flag == 1:
                data = np.zeros((dat.shape[0], dat.shape[1]))
                data = data + dat
                labels = np.zeros((lab.shape[0]))
                labels = labels + lab
                flag = 0
            else:
                data = np.append(data,dat, axis=0)
                labels = np.append(labels, lab, axis=0)
        return (data,labels)
    else:
        return getData(file)

 
def L1(X,y,W):
    delta = 1.0                                                                         # Defined delta value
    y = y.reshape(labels.shape[0],1)
    score = W.dot(X.T)                                                                  # Calculating Score 
    correct_class_score = score[np.array(y[:,0].astype(np.uint64)),np.array(np.arange(X.T.shape[1]))]     # Getting correct class score
    margins = np.maximum(0,(score - correct_class_score + delta))                       # Calculating loss 
    margins[[y[:,0].astype(np.uint64)],[np.arange(X.T.shape[1])]] = 0                                     # Setting the loss to 0 for correct class
    loss = np.sum(margins,axis=0)
    return np.mean(loss)

def CIFAR10_loss(W):
    global data, labels
    return L1(data,labels.reshape(labels.shape[0],1),W)

if __name__ == '__main__':
    global data, labels
    trainFile = ['/home/kpit/Images/data_batch_1']#, '/home/kpit/Images/data_batch_2', '/home/kpit/Images/data_batch_3', '/home/kpit/Images/data_batch_4', '/home/kpit/Images/data_batch_5']
    testFile = '/home/kpit/Images/test_batch'
    data_here, labels_here = load_CIFAR10(trainFile)                 # function to load the dataset
    Xte, Yte = load_CIFAR10(testFile)
    W = np.random.randn(10, 3072) * 0.001                            # generate random starting W
    bestloss = float("inf")
    for i in range(10):
      step_size = 0.0001
      Wtry = W + np.random.randn(10, 3072) * step_size
      start_index = np.random.randint(0,9949)
      data = data_here[start_index:start_index+50,:]                 # Taking out a mini-batch from whole data set for finding gradient
      labels = labels_here[start_index:start_index+50]
      loss = CIFAR10_loss(Wtry)
      if loss < bestloss:                                            # If the new calculated loss is less than previous loss
        W = Wtry                                                     # use the new weight matrixb
        bestloss = loss
      print('iter {} loss is {}'.format(i, bestloss))

**Strategy #3: Following the Gradient**
In the previous section we tried to find a direction in the weight-space that would improve our weight vector (and give us a lower loss). It turns out that there is no need to randomly search for a good direction: we can compute the best direction along which we should change our weight vector that is mathematically guaranteed to be the direction of the steepest descend (at least in the limit as the step size goes towards zero). This direction will be related to the **gradient** of the loss function. In our hiking analogy, this approach roughly corresponds to feeling the slope of the hill below our feet and stepping down the direction that feels steepest.

In one-dimensional functions, the slope is the instantaneous rate of change of the function at any point you might be interested in. The gradient is a generalization of slope for functions that don’t take a single number but a vector of numbers. Additionally, the gradient is just a vector of slopes (more commonly referred to as **derivatives**) for each dimension in the input space. The mathematical expression for the derivative of a 1-D function with respect its input is:

$$ \frac{df(x)}{dx} = \lim_{h\ \to 0} \frac{f(x + h) - f(x)}{h} $$

When the functions of interest take a vector of numbers instead of a single number, we call the derivatives partial derivatives, and the gradient is simply the vector of **partial derivatives** in each dimension.

![Matrix Derivative](Images/matrixDerivative.png)

![Gradient Descent](Images/gradient_descent.png)

In [ ]:
import random
data = np.array([]) 
labels = np.array([])

def CIFAR10_loss(W):
    global data, labels
    return L1(data,labels.reshape(labels.shape[0],1),W)

def eval_gradient(f,W):                                                         # Mini-batch gradient descent
    fx = f(W)
    grad = np.zeros(W.shape)
    h = 0.00001
    iterator = np.nditer(W,flags=['multi_index'],op_flags=['readwrite'])
    while not iterator.finished:
        iw = iterator.multi_index
        old_value = W[iw]
        W[iw] = old_value + h
        fxh = f(W)
        W[iw] = old_value
        grad[iw] = np.mean((fxh - fx)/h)
        iterator.iternext()
    return grad

if __name__ == '__main__':
    global data, labels
    file_name ='Datasets/data_batch_2'
    data_here, labels_here = loadData(file_name)                                # Loading data
    bias = np.zeros((10,1)) + 1                                                 # bias vector
    bias_data = np.tile(bias,(1000,1))
    data_here = np.append(data_here,bias_data,axis=1)
    weight = np.append((np.random.rand(10,3072)*0.001),bias,axis=1)             # performing "Bias Trick"
    for i in range(700):
        start_index = random.randint(0,9949)
        data = data_here[start_index:start_index+50,:]                          # Taking out a mini-batch from whole data set for finding gradient
        labels = labels_here[start_index:start_index+50]                        
        df = eval_gradient(CIFAR10_loss,weight)                                 # Evaluating gradient
        weight = weight - (10**-8) * df                                         # adjusting weight by taking a small step in direction of gradient
        print('Accuracy: {}'.format(Test_Weight(data_here,labels_here,weight))) # printing accuracy
    
    file_pointer = open('Datasets/weightMatrix','wb')
    pickle.dump(weight,file_pointer)                                            # Saving calculated weight matrix
    file_name ='Datasets/test_batch'
    data, labels = loadData(file_name)
    data = np.append(data,bias_data,axis=1)
    print('Accuracy result: {}'.format(Test_Weight(data,labels,weight)))        # Testing the calcualted weight  
    

![](Images\grad.png)

**Update in negative gradient direction:** In the code above, notice that to compute W_new we are making an update in the negative direction of the gradient df since we wish our loss function to decrease, not increase.

**Effect of step size:** The gradient tells us the direction in which the function has the steepest rate of increase, but it does not tell us how far along this direction we should step. As we will see later in the course, choosing the step size (also called the learning rate) will become one of the most important (and most headache-inducing) hyperparameter settings in training a neural network. In our blindfolded hill-descent analogy, we feel the hill below our feet sloping in some direction, but the step length we should take is uncertain. If we shuffle our feet carefully we can expect to make consistent but very small progress (this corresponds to having a small step size). Conversely, we can choose to make a large, confident step in an attempt to descend faster, but this may not pay off. As you can see in the code example above, at some point taking a bigger step gives a higher loss as we “**overstep**”.

### Gradient Descent

Now that we can compute the gradient of the loss function, the procedure of repeatedly evaluating the gradient and then performing a parameter update is called Gradient Descent.

In [ ]:
# Vanilla Gradient Descent

while True:
  weights_grad = evaluate_gradient(loss_fun, data, weights)                # Evaluating Gradient
  weights += - step_size * weights_grad                                    # perform parameter update by taking a small step in the direction of gradient

**Mini-batch gradient descent:** In large-scale applications, the training data can have on order of millions of examples. Hence, it seems wasteful to compute the full loss function over the entire training set in order to perform only a single parameter update. A very common approach to addressing this challenge is to compute the gradient over batches of the training data. This batch is then used to perform a parameter update.

In [ ]:
# Vanilla Minibatch Gradient Descent

while True:
  data_batch = sample_training_data(data, 256)                             # sample 256 examples
  weights_grad = evaluate_gradient(loss_fun, data_batch, weights)          # Evaluating Gradient 
  weights += - step_size * weights_grad                                    # perform parameter update

The extreme case of this is a setting where the mini-batch contains only a single example. This process is called **Stochastic Gradient Descent (SGD)** (or also sometimes on-line gradient descent). This is relatively less common to see because in practice due to vectorized code optimizations it can be computationally much more efficient to evaluate the gradient for 100 examples, than the gradient for one example 100 times.

To summarize the thing we just learned, here a data-flow diagram-

![Data Flow Diagram](Images/dataflow.jpeg)

### Simple expressions and interpretation of the gradient

Lets start simple so that we can develop the notation and conventions for more complex expressions. Consider a simple multiplication function of two numbers $f(x,y)=xy$. It is a matter of simple calculus to derive the partial derivative for either input:

$$ f(x,y) = x y \hspace{0.5in} \rightarrow \hspace{0.5in} \frac{\partial f}{\partial x} = y \hspace{0.5in} \frac{\partial f}{\partial y} = x $$

**Interpretation:** Keep in mind what the derivatives tell you: They indicate the rate of change of a function with respect to that variable surrounding an infinitesimally small region near a particular point:

$$ \frac{df(x)}{dx} = \lim_{h\ \to 0} \frac{f(x + h) - f(x)}{h} $$

A technical note is that the division sign on the left-hand side is, unlike the division sign on the right-hand side, not a division. Instead, this notation indicates that the operator $\frac{d}{dx}$ is being applied to the function $f$, and returns a different function (the derivative). A nice way to think about the expression above is that when $h$ is very small, then the function is well-approximated by a straight line, and the derivative is its slope. In other words, the derivative on each variable tells you the sensitivity of the whole expression on its value. For example, if $x=4,y=−3$ then $f(x,y)=−12$ and the derivative on x $\frac{∂f}{∂x}=−3$. This tells us that if we were to increase the value of this variable by a tiny amount, the effect on the whole expression would be to decrease it (due to the negative sign), and by three times that amount. This can be seen by rearranging the above equation ( $f(x+h)=f(x)+h\frac{df(x)}{dx}$ ). Analogously, since $\frac{∂f}{∂y}=4$, we expect that increasing the value of $y$ by some very small amount $h$ would also increase the output of the function (due to the positive sign), and by $4h$.

> The derivative on each variable tells you the sensitivity of the whole expression on its value.

### Compound expressions with chain rule

Lets now start to consider more complicated expressions that involve multiple composed functions, such as $f(x,y,z)=(x+y)z$. This expression is still simple enough to differentiate directly, but we’ll take a particular approach to it that will be helpful with understanding the intuition behind backpropagation. In particular, note that this expression can be broken down into two expressions: $q=x+y$ and $f=qz$. Moreover, we know how to compute the derivatives of both expressions separately, as seen in the previous section. $f$ is just multiplication of $q$ and $z$, so $\frac{∂f}{∂q}=z,\frac{∂f}{∂z}=q$, and $q$ is addition of $x$ and $y$ so $\frac{∂q}{∂x}=1,\frac{∂q}{∂y}=1$. However, we don’t necessarily care about the gradient on the intermediate value $q$ - the value of $\frac{∂f}{∂q}$ is not useful. Instead, we are ultimately interested in the gradient of $f$ with respect to its inputs $x,y,z$. The chain rule tells us that the correct way to “chain” these gradient expressions together is through multiplication. For example, $\frac{∂f}{∂x}=\frac{∂f}{∂q}\frac{∂q}{∂x}$. In practice this is simply a multiplication of the two numbers that hold the two gradients. Lets see this with an example:

In [ ]:
# set some inputs
x = -2; y = 5; z = -4

# perform the forward pass
q = x + y # q becomes 3
f = q * z # f becomes -12

# perform the backward pass (backpropagation) in reverse order:
# first backprop through f = q * z
dfdz = q # df/dz = q, so gradient on z becomes 3
dfdq = z # df/dq = z, so gradient on q becomes -4
# now backprop through q = x + y
dfdx = 1.0 * dfdq # dq/dx = 1. And the multiplication here is the chain rule!
dfdy = 1.0 * dfdq # dq/dy = 1

This computation can also be nicely visualized with a circuit diagram:
 
![Chain Rule](Images\chainRule.png)

### Intuitive understanding of backpropagation

Notice that backpropagation is a beautifully local process. Every gate in a circuit diagram gets some inputs and can right away compute two things: 1. its output value and 2. the local gradient of its inputs with respect to its output value. Notice that the gates can do this completely **independently** without being aware of any of the details of the full circuit that they are embedded in.

Lets get an intuition for how this works by referring again to the example. The add gate received inputs [-2, 5] and computed output 3. Since the gate is computing the addition operation, its local gradient for both of its inputs is +1. The rest of the circuit computed the final value, which is -12. During the backward pass in which the chain rule is applied recursively backwards through the circuit, the add gate (which is an input to the multiply gate) learns that the gradient for its output was -4. If we anthropomorphize the circuit as wanting to output a higher value (which can help with intuition), then we can think of the circuit as “wanting” the output of the add gate to be lower (due to negative sign), and with a force of 4. To continue the recurrence and to chain the gradient, the add gate takes that gradient and multiplies it to all of the local gradients for its inputs (making the gradient on both x and y 1 $\times$ -4 = -4). Notice that this has the desired effect: If **x,y** were to decrease (responding to their negative gradient) then the add gate’s output would decrease, which in turn makes the multiply gate’s output increase.

Backpropagation can thus be thought of as gates communicating to each other (through the gradient signal) whether they want their outputs to increase or decrease (and how strongly), so as to make the final output value higher.

### Modularity: Sigmoid 
The gates we introduced above are relatively arbitrary. Any kind of differentiable function can act as a gate, and we can group multiple gates into a single gate, or decompose a function into multiple gates whenever it is convenient. Lets look at another expression that illustrates this point:

$$ f(w,x) = \frac{1}{1+e^{-(w_0x_0 + w_1x_1 + w_2)}} $$

 this expression describes a 2-dimensional neuron (with inputs x and weights w) that uses the sigmoid activation function. But for now lets think of this very simply as just a function from inputs w,x to a single number. The function is made up of multiple gates. In addition to the ones described already above (add, mul, max), there are four more:
 
$$
f(x) = \frac{1}{x} 
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = -1/x^2 
\\\\
f_c(x) = c + x
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = 1 
\\\\
f(x) = e^x
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = e^x
\\\\
f_a(x) = ax
\hspace{1in} \rightarrow \hspace{1in} 
\frac{df}{dx} = a
$$

Where the functions $f_c,f_a$ translate the input by a constant of $c$ and scale the input by a constant of $a$, respectively. These are technically special cases of addition and multiplication, but we introduce them as (new) unary gates here since we do need the gradients for the constants. $c,a$. The full circuit then looks as follows:

![Circuit Diagram](Images/circuit.png)

In the example above, we see a long chain of function applications that operates on the result of the dot product between w,x. The function that these operations implement is called the sigmoid function $σ(x)$. It turns out that the derivative of the sigmoid function with respect to its input simplifies if you perform the derivation (after a fun tricky part where we add and subtract a 1 in the numerator):

$$
\sigma(x) = \frac{1}{1+e^{-x}} \\\\
\rightarrow \hspace{0.3in} \frac{d\sigma(x)}{dx} = \frac{e^{-x}}{(1+e^{-x})^2} = \left( \frac{1 + e^{-x} - 1}{1 + e^{-x}} \right) \left( \frac{1}{1+e^{-x}} \right) 
= \left( 1 - \sigma(x) \right) \sigma(x)
$$

As we see, the gradient turns out to simplify and becomes surprisingly simple. For example, the sigmoid expression receives the input 1.0 and computes the output 0.73 during the forward pass. The derivation above shows that the _local_ gradient would simply be (1 - 0.73) * 0.73 ~= 0.2, as the circuit computed before (see the image above), except this way it would be done with a single, simple and efficient expression (and with less numerical issues). Therefore, in any real practical application it would be very useful to group these operations into a single gate. Lets see the backprop for this neuron in code:

In [ ]:
w = [2,-3,-3] # assume some random weights and data
x = [-1, -2]

# forward pass
dot = w[0]*x[0] + w[1]*x[1] + w[2]
f = 1.0 / (1 + math.exp(-dot)) # sigmoid function

# backward pass through the neuron (backpropagation)
ddot = (1 - f) * f # gradient on dot variable, using the sigmoid gradient derivation
dx = [w[0] * ddot, w[1] * ddot] # backprop into x
dw = [x[0] * ddot, x[1] * ddot, 1.0 * ddot] # backprop into w
# we're done! we have the gradients on the inputs to the circuit

**Gradients add up at forks:** The forward expression involves the variables x,y multiple times, so when we perform backpropagation we must be careful to use **+=** instead of **=** to accumulate the gradient on these variables (otherwise we would overwrite it). This follows the multivariable chain rule in Calculus, which states that if a variable branches out to different parts of the circuit, then the gradients that flow back to it will add.

The **add gate** always takes the gradient on its output and distributes it equally to all of its inputs, regardless of what their values were during the forward pass. This follows from the fact that the local gradient for the add operation is simply +1.0, so the gradients on all inputs will exactly equal the gradients on the output because it will be multiplied by x1.0 (and remain unchanged). In the example circuit above, note that the + gate routed the gradient of 2.00 to both of its inputs, equally and unchanged.

The **max gate** routes the gradient. Unlike the add gate which distributed the gradient unchanged to all its inputs, the max gate distributes the gradient (unchanged) to exactly one of its inputs (the input that had the highest value during the forward pass). This is because the local gradient for a max gate is 1.0 for the highest value, and 0.0 for all other values. In the example circuit above, the max operation routed the gradient of 2.00 to the z variable, which had a higher value than w, and the gradient on w remains zero.

The **multiply gate** is a little less easy to interpret. Its local gradients are the input values (except switched), and this is multiplied by the gradient on its output during the chain rule. In the example above, the gradient on x is -8.00, which is -4.00 x 2.00.

**Unintuitive effects and their consequences:** Notice that if one of the inputs to the multiply gate is very small and the other is very big, then the multiply gate will do something slightly unintuitive: it will assign a relatively huge gradient to the small input and a tiny gradient to the large input. Note that in linear classifiers where the weights are dot producted $w^Tx_i$ (multiplied) with the inputs, this implies that the scale of the data has an effect on the magnitude of the gradient for the weights. For example, if you multiplied all input data examples $x_i$ by 1000 during preprocessing, then the gradient on the weights will be 1000 times larger, and you’d have to lower the learning rate by that factor to compensate. This is why preprocessing matters a lot, sometimes in subtle ways! And having intuitive understanding for how the gradients flow can help you debug some of these cases.

### Modelling a neuron

| | |
| :---: | :---: |
|![Biological Neuron](Images/neuron.png)|![Neuron Mathematical Model](Images/neuron_model.jpeg) |

each neuron performs a dot product with the input and its weights, adds the bias and applies the non-linearity (or activation function), in this case the sigmoid $σ(x)=1/(1+e^−x)$. The mathematical form of the model Neuron’s forward computation might look familiar to you. As we saw with linear classifiers, a neuron has the capacity to “like” (activation near one) or “dislike” (activation near zero) certain linear regions of its input space. Hence, with an appropriate loss function on the neuron’s output, we can turn a single neuron into a linear classifier.

### Commonly used activation functions

Every activation function (or non-linearity) takes a single number and performs a certain fixed mathematical operation on it. There are several activation functions you may encounter in practice:

| | |
| :---: | :---: |
|![Sigmoid Function](Images/sigmoid.jpeg)|![Tanh Function](Images/tanh.jpeg) |
Left: Sigmoid non-linearity squashes real numbers to range between [0,1] Right: The tanh non-linearity squashes real numbers to range between [-1,1].

**Sigmoid:** The sigmoid non-linearity has the mathematical form $σ(x)=1/(1+e^−x)$ and is shown in the image above on the left. As alluded to in the previous section, it takes a real-valued number and “squashes” it into range between 0 and 1. In particular, large negative numbers become 0 and large positive numbers become 1. The sigmoid function has seen frequent use historically since it has a nice interpretation as the firing rate of a neuron: from not firing at all (0) to fully-saturated firing at an assumed maximum frequency (1). In practice, the sigmoid non-linearity has recently fallen out of favor and it is rarely ever used. It has two major drawbacks:
* **Sigmoids saturate and kill gradients:** A very undesirable property of the sigmoid neuron is that when the neuron’s activation saturates at either tail of 0 or 1, the gradient at these regions is almost zero. Recall that during backpropagation, this (local) gradient will be multiplied to the gradient of this gate’s output for the whole objective. Therefore, if the local gradient is very small, it will effectively “kill” the gradient and almost no signal will flow through the neuron to its weights and recursively to its data. Additionally, one must pay extra caution when initializing the weights of sigmoid neurons to prevent saturation. For example, if the initial weights are too large then most neurons would become saturated and the network will barely learn.
* **Sigmoid outputs are not zero-centered:** This is undesirable since neurons in later layers of processing in a Neural Network (more on this soon) would be receiving data that is not zero-centered. This has implications on the dynamics during gradient descent, because if the data coming into a neuron is always positive (e.g. $x>0$ elementwise in $f=w^Tx+b$)), then the gradient on the weights $w$ will during backpropagation become either all be positive, or all negative (depending on the gradient of the whole expression ff). This could introduce undesirable zig-zagging dynamics in the gradient updates for the weights. However, notice that once these gradients are added up across a batch of data the final update for the weights can have variable signs, somewhat mitigating this issue. Therefore, this is an inconvenience but it has less severe consequences compared to the saturated activation problem above.

**Tanh:** The tanh non-linearity is shown on the image above on the right. It squashes a real-valued number to the range [-1, 1]. Like the sigmoid neuron, its activations saturate, but unlike the sigmoid neuron its output is zero-centered. Therefore, in practice the tanh non-linearity is always preferred to the sigmoid nonlinearity. Also note that the tanh neuron is simply a scaled sigmoid neuron, in particular the following holds: $tanh(x)=2σ(2x)−1$.

| | |
| :---: | :---: |
|![Rectified Linear Unit](Images/relu.jpeg)|![Convergence Plot(ReLu VS Tanh)](Images/alexplot.jpeg) |
**Left:** Rectified Linear Unit (ReLU) activation function, which is zero when x < 0 and then linear with slope 1 when x > 0. **Right:** A plot from Krizhevsky et al. (pdf) paper indicating the 6x improvement in convergence with the ReLU unit compared to the tanh unit.

**ReLU:** The Rectified Linear Unit has become very popular in the last few years. It computes the function $f(x)=max(0,x)$. In other words, the activation is simply thresholded at zero (see image above on the left). There are several pros and cons to using the ReLUs:
* (+) It was found to greatly accelerate the convergence of stochastic gradient descent compared to the sigmoid/tanh functions. It is argued that this is due to its linear, non-saturating form.
* (+) Compared to tanh/sigmoid neurons that involve expensive operations (exponentials, etc.), the ReLU can be implemented by simply thresholding a matrix of activations at zero.
* (-) Unfortunately, ReLU units can be fragile during training and can “die”. For example, a large gradient flowing through a ReLU neuron could cause the weights to update in such a way that the neuron will never activate on any datapoint again. If this happens, then the gradient flowing through the unit will forever be zero from that point on. That is, the ReLU units can irreversibly die during training since they can get knocked off the data manifold. For example, you may find that as much as **40% of your network can be “dead”** (i.e. neurons that never activate across the entire training dataset) if the learning rate is set too high. With a proper setting of the learning rate this is less frequently an issue.

**Leaky ReLU:** Leaky ReLUs are one attempt to fix the “dying ReLU” problem. Instead of the function being zero when x < 0, a leaky ReLU will instead have a small negative slope (of 0.01, or so). That is, the function computes $f(x)=𝟙(x<0)(αx)+𝟙(x>=0)(x)$ where $α$ is a small constant.

**Maxout:** Other types of units have been proposed that do not have the functional form $f(w^Tx+b)$ where a non-linearity is applied on the dot product between the weights and the data. One relatively popular choice is the Maxout neuron that generalizes the ReLU and its leaky version. The Maxout neuron computes the function $max(w^T_1x+b_1,w^T_2x+b_2)$. Notice that both ReLU and Leaky ReLU are a special case of this form (for example, for ReLU we have $w1,b1=0$). The Maxout neuron therefore enjoys all the benefits of a ReLU unit (linear regime of operation, no saturation) and does not have its drawbacks (dying ReLU). However, unlike the ReLU neurons it doubles the number of parameters for every single neuron, leading to a high total number of parameters.

### Neural Network architectures
#### Layer-wise organization

**Neural Networks as neurons in graphs:** Neural Networks are modeled as collections of neurons that are connected in an acyclic graph. In other words, the outputs of some neurons can become inputs to other neurons. Cycles are not allowed since that would imply an infinite loop in the forward pass of a network. Instead of an amorphous blobs of connected neurons, Neural Network models are often organized into distinct layers of neurons. For regular neural networks, the most common layer type is the **fully-connected layer** in which neurons between two adjacent layers are fully pairwise connected, but neurons within a single layer share no connections. Below are two example Neural Network topologies that use a stack of fully-connected layers:

| | |
| :---: | :---: |
|![2-layer Neural Network](Images/neural_net_2.jpeg)|![3-layer Neural Network](Images/neural_net_3.jpeg) |
**Left:** A 2-layer Neural Network (one hidden layer of 4 neurons (or units) and one output layer with 2 neurons), and three inputs. **Right:** A 3-layer neural network with three inputs, two hidden layers of 4 neurons each and one output layer. Notice that in both cases there are connections (synapses) between neurons across layers, but not within a layer.

**Naming conventions:** Notice that when we say N-layer neural network, we do not count the input layer. Therefore, a single-layer neural network describes a network with no hidden layers (input directly mapped to output). In that sense, you can sometimes hear people say that logistic regression or SVMs are simply a special case of single-layer Neural Networks. Many people do not like the analogies between Neural Networks and real brains and prefer to refer to neurons as units.

**Output layer:** Unlike all layers in a Neural Network, the output layer neurons most commonly do not have an activation function (or you can think of them as having a linear identity activation function). This is because the last output layer is usually taken to represent the class scores (e.g. in classification), which are arbitrary real-valued numbers, or some kind of real-valued target (e.g. in regression).

### Feed-forward computation

Repeated matrix multiplications interwoven with activation function. One of the primary reasons that Neural Networks are organized into layers is that this structure makes it very simple and efficient to evaluate Neural Networks using matrix vector operations. Working with the example three-layer neural network in the diagram above, the input would be a [3x1] vector. All connection strengths for a layer can be stored in a single matrix. For example, the first hidden layer’s weights W1 would be of size [4x3], and the biases for all units would be in the vector b1, of size [4x1]. Here, every single neuron has its weights in a row of W1, so the matrix vector multiplication np.dot(W1,x) evaluates the activations of all neurons in that layer. Similarly, W2 would be a [4x4] matrix that stores the connections of the second hidden layer, and W3 a [1x4] matrix for the last (output) layer. The full forward pass of this 3-layer neural network is then simply three matrix multiplications, interwoven with the application of the activation function:

In [ ]:
# forward-pass of a 3-layer neural network:
f = lambda x: 1.0/(1.0 + np.exp(-x))            # activation function (use sigmoid)
x = np.random.randn(3, 1)                       # random input vector of three numbers (3x1)
h1 = f(np.dot(W1, x) + b1)                      # calculate first hidden layer activations (4x1)
h2 = f(np.dot(W2, h1) + b2)                     # calculate second hidden layer activations (4x1)
out = np.dot(W3, h2) + b3                       # output neuron (1x1)

### Setting number of layers and their sizes

How do we decide on what architecture to use when faced with a practical problem? Should we use no hidden layers? One hidden layer? Two hidden layers? How large should each layer be? First, note that as we increase the size and number of layers in a Neural Network, the **capacity** of the network increases. That is, the space of representable functions grows since the neurons can collaborate to express many different functions. For example, suppose we had a binary classification problem in two dimensions. We could train three separate neural networks, each with one hidden layer of some size and obtain the following classifiers:

![ Hidden Units ](Images/layer_sizes20.jpeg) 
In the diagram above, we can see that Neural Networks with more neurons can express more complicated functions. However, this is both a blessing (since we can learn to classify more complicated data) and a curse (since it is easier to overfit the training data). **Overfitting** occurs when a model with high capacity fits the noise in the data instead of the (assumed) underlying relationship. For example, the model with 20 hidden neurons fits all the training data but at the cost of segmenting the space into many disjoint red and green decision regions. The model with 3 hidden neurons only has the representational power to classify the data in broad strokes. It models the data as two blobs and interprets the few red points inside the green cluster as **outliers** (noise). In practice, this could lead to better **generalization** on the test set.

Based on our discussion above, it seems that smaller neural networks can be preferred if the data is not complex enough to prevent overfitting. However, this is incorrect - there are many other preferred ways to prevent overfitting in Neural Networks. In practice, it is always better to use these methods to control overfitting instead of the number of neurons. In practice, what you find is that if you train a small network the final loss can display a good amount of variance - in some cases you get lucky and converge to a good place but in some cases you get trapped in one of the bad minima. On the other hand, if you train a large network you’ll start to find many different solutions, but the variance in the final achieved loss will be much smaller. In other words, all solutions are about equally as good, and rely less on the luck of random initialization. To reiterate, the regularization strength is the preferred way to control the overfitting of a neural network. We can look at the results achieved by three different settings:

![Regularization Parameters](Images/reg_strengths.jpeg)

The takeaway is that you should not be using smaller networks because you are afraid of overfitting. Instead, you should use as big of a neural network as your computational budget allows, and use other regularization techniques to control overfitting.

### The BIGGER the better

#### Local Minima
Sometimes your loss functions has a funny shape, and following the slope doesn't take you to the absolute lowest point. Consider the picture below.

| | |
| :---: | :---: |
|![Problem of Local Minima](Images/sgd_local_min.png)| ![Optimization Function](Images/sgd_randomness_ensemble.png) |

This is by far the **most difficult problem with gradient descent**. There are a myriad of options to try to overcome this. Generally speaking, they all involve an element of random searching to try lots of different parts of the space. Now we are ready to the question why using more number of units perform better.

Imagine that we randomly placed 100 balls on this line and started optimizing all of them. If we did so, they would all end up in only 5 positions, mapped out by the five colored balls above. The colored regions represent the domain of each local minimum. For example, if a ball randomly falls within the blue domain, it will converge to the blue minimum. This means that to search the entire space, we only have to randomly find 5 spaces! This is far better than pure random searching, which has to randomly try every space (which could easily be millions of places on this black line depending on the granularity).

**In Neural Networks:** One way that neural networks accomplish this is by having very large hidden layers. You see, each hidden node in a layer starts out in a different random starting state. This allows each hidden node to converge to different patterns in the network. Parameterizing this size allows the neural network user to potentially try thousands (or tens of billions) of different local minima in a single neural network.


### Example: A small scale neural network  
Now that we have learnt all the bits and pieces, lets put it together to understand the whole process. We will start simple, by mimicing a simple **XOR-gate** operation. So out input is two values(either 1 or 0) and the produced result is, XOR of the values pluged in. We will randomly generate these values and the associated weight matrix( weight matrix are also called synapse by a lot of people, so I will use both the terms interchangebly).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

no_example = 10                               # Setting number of examples
# Generating random input data
X = np.c_[np.random.randint(0,2,no_example).reshape(-1,1),np.random.randint(0,2,no_example).reshape(-1,1)]
    
# Generating output data    
y = np.logical_and(X[:,0], X[:,1]).reshape(-1,1).astype(np.float64)

lossArr = []                                  # list to store values for plotting

syn0 = 2*np.random.random((2,5)) - 1          # synapse_1 of size 2 X 5
syn1 = 2*np.random.random((5,1)) - 1          # synapse_2 of size 5 X 1

# Learing Phase
for j in range(20000):
    l1 = 1/(1+np.exp(-(np.dot(X,syn0))))      # applying sigmoid to input X synapse_1 
    l2 = 1/(1+np.exp(-(np.dot(l1,syn1))))     # applying sigmoid to layer_1 X synapse_2
    loss = np.sum(l2-y)                       # Calculated loss
    if j < 200:
        lossArr.append(loss)
    if j%1000==0:
        print(loss)
    l2_delta = (y - l2)*(l2*(1-l2))                      # layer_2_delta = loss_delta * sigmoid_delta
    l1_delta = l2_delta.dot(syn1.T) * (l1 * (1-l1))      # layer_1_delta = layer_2 dealta * sigmoid_delta
    syn1 += l1.T.dot(l2_delta)                           # Adjusting weights in synapse_1  
    syn0 += X.T.dot(l1_delta)                            # Adjusting weights in synapse_2
    
print("Accuracy: {}".format(np.mean(np.equal(y,np.round(l2)))))      # Printing out the accuracy of the model
# Plotting out values on a plot
plt.plot(lossArr,'r-')                                           
plt.show()

# Convolutional Neural Networks
Convolutional Neural Networks are very similar to ordinary Neural Networks as they are made up of neurons that have learnable weights and biases. ConvNet architectures make the explicit assumption that the inputs are Images, which allows us to encode certain properties into the architecture. These then make the forward function more efficient to implement and vastly reduce the amount of parameters in the network.

**3D volumes of neurons:** Convolutional Neural Networks take advantage of the fact that the input consists of Images and they constrain the architecture in a more sensible way. In particular, unlike a regular Neural Network, the layers of a ConvNet have neurons arranged in 3 dimensions: width, height, depth. (Note that the word depth here refers to the third dimension of an activation volume, not to the depth of a full Neural Network, which can refer to the total number of layers in a network.) For example, the input Images in CIFAR-10 are an input volume of activations, and the volume has dimensions 32x32x3 (width, height, depth respectively). As we will soon see, the neurons in a layer will only be connected to a small region of the layer before it, instead of all of the neurons in a fully-connected manner. Moreover, the final output layer would for CIFAR-10 have dimensions 1x1x10, because by the end of the ConvNet architecture we will reduce the full image into a single vector of class scores, arranged along the depth dimension. Here is a visualization:

![Convolutional Neural Network](Images/cnn.jpeg)

## Layers used to build ConvNets
* **INPUT:** [32x32x3] will hold the raw pixel values of the image, in this case an image of width 32, height 32, and with three color channels R,G,B.
* **CONV layer** will compute the output of neurons that are connected to local regions in the input, each computing a dot product between their weights and a small region they are connected to in the input volume. This may result in volume such as [32x32x12] if we decided to use 12 filters.
* **RELU layer** will apply an elementwise activation function, such as the max(0,x)max(0,x) thresholding at zero. This leaves the size of the volume unchanged ([32x32x12]).
* **POOL layer** will perform a downsampling operation along the spatial dimensions (width, height), resulting in volume such as [16x16x12].
* **FC (i.e. fully-connected) layer:** will compute the class scores, resulting in volume of size [1x1x10], where each of the 10 numbers correspond to a class score, such as among the 10 categories of CIFAR-10. As with ordinary Neural Networks and as the name implies, each neuron in this layer will be connected to all the numbers in the previous volume.

Now we will see each of these layers in a bit more detail.

### Convolution Layer
 The CONV layer’s parameters consist of a set of learnable filters. Every filter is small spatially (along width and height), but extends through the full depth of the input volume. During the forward pass, we slide (more precisely, convolve) each filter across the width and height of the input volume and compute dot products between the entries of the filter and the input at any position.  **Intuitively, the network will learn filters that activate when they see some type of visual feature such as an edge** of some orientation or a blotch of some color on the first layer, or eventually entire honeycomb or wheel-like patterns on higher layers of the network. Now, we will have an entire set of filters in each CONV layer (e.g. 12 filters), and each of them will produce a separate 2-dimensional activation map. We will stack these activation maps along the depth dimension and produce the output volume.
 
**Local Connectivity:** When dealing with high-dimensional inputs such as Images, as we saw above it is impractical to connect neurons to all neurons in the previous volume. Instead, we will connect each neuron to only a local region of the input volume. The spatial extent of this connectivity is a hyperparameter called the receptive field of the neuron (equivalently this is the filter size). The extent of the connectivity along the depth axis is always equal to the depth of the input volume. Below is an example input volume in red (e.g. a 32x32x3 CIFAR-10 image), and an example volume of neurons in the first Convolutional layer. Each neuron in the convolutional layer is connected only to a local region in the input volume spatially, but to the full depth

![Convolution Operation](Images/depthcol.jpeg)

**Spatial arrangement:**We will now discuss the three hyperparameters that controls the size of the output volume: the depth, stride and zero-padding.
1. First, the **depth** of the output volume is a hyperparameter: it corresponds to the number of filters we would like to use, each learning to look for something different in the input. For example, if the first Convolutional Layer takes as input the raw image, then different neurons along the depth dimension may activate in presence of various oriented edges, or blobs of color. We will refer to a set of neurons that are all looking at the same region of the input as a depth column (some people also prefer the term fibre).
2. Second, we must specify the **stride** with which we slide the filter. When the stride is 1 then we move the filters one pixel at a time. When the stride is 2 (or uncommonly 3 or more, though this is rare in practice) then the filters jump 2 pixels at a time as we slide them around. This will produce smaller output volumes spatially.
3. As we will soon see, sometimes it will be convenient to pad the input volume with zeros around the border. The size of this **zero-padding** is a hyperparameter. The nice feature of zero padding is that it will allow us to control the spatial size of the output volumes.

We can compute the spatial size of the output volume as a function of the input volume size ($W$), the receptive field size of the Conv Layer neurons ($F$), the stride with which they are applied ($S$), and the amount of zero padding used ($P$) on the border. You can convince yourself that the correct formula for calculating how many neurons “fit” is given by $(W−F+2P)/S+1$. For example for a 7x7 input and a 3x3 filter with stride 1 and pad 0 we would get a 5x5 output. With stride 2 we would get a 3x3 output. Lets also see one more graphical example, in the below example there is only one spatial dimension (x-axis), one neuron with a receptive field size of F = 3, the input size is W = 5, and there is zero padding of P = 1. **Left:** The neuron strided across the input in stride of S = 1, giving output of size (5 - 3 + 2)/1+1 = 5. **Right:** The neuron uses stride of S = 2, giving output of size (5 - 3 + 2)/2+1 = 3.

![Output Dimention Calculation](Images/stride.jpeg)

**Parameter Sharing:**Parameter sharing scheme is used in Convolutional Layers to control the number of parameters. Using the real-world example above, we see that there are $ 55 \times 55 \times 96 = 290 $,400 neurons in the first Conv Layer, and each has 11 $\times$ 11 $\times$ 3 = 363 weights and 1 bias. Together, this adds up to 290400 $\times$ 364 = **105,705,600 parameters** on the first layer of the ConvNet alone. Clearly, this number is very high. It turns out that we can dramatically reduce the number of parameters by making one reasonable assumption: That if one feature is useful to compute at some spatial position (x,y), then it should also be useful to compute at a different position (x2,y2). In other words, denoting a single 2-dimensional slice of depth as a depth slice (e.g. a volume of size [55x55x96] has 96 depth slices, each of size [55x55]), we are going to constrain the neurons in each depth slice to use the same weights and bias. With this parameter sharing scheme, the first Conv Layer in our example would now have only 96 unique set of weights (one for each depth slice), for a total of 96x11x11x3 = 34,848 unique weights, or **34,944 parameters** (+96 biases).

Notice that if all neurons in a single depth slice are using the same weight vector, then the forward pass of the CONV layer can in each depth slice be computed as a convolution of the neuron’s weights with the input volume (Hence the name: Convolutional Layer). This is why it is common to refer to the sets of weights as a filter (or a kernel), that is convolved with the input.

![Learned Weights(ALexnet)](Images/weights.jpeg)

Example filters learned by Krizhevsky et al. Each of the 96 filters shown here is of size [11x11x3], and each one is shared by the 55$\times$55 neurons in one depth slice. Notice that the parameter sharing assumption is relatively reasonable: If detecting a horizontal edge is important at some location in the image, it should intuitively be useful at some other location as well due to the translationally-invariant structure of Images. There is therefore no need to relearn to detect a horizontal edge at every one of the 55$\times$55 distinct locations in the Conv layer output volume.

#### Convolution Implementation as Matrix Multiplication
Implementation as Matrix Multiplication. Note that the convolution operation essentially performs dot products between the filters and local regions of the input. A common implementation pattern of the CONV layer is to take advantage of this fact and formulate the forward pass of a convolutional layer as one big matrix multiply as follows:

1. The local regions in the input image are stretched out into columns in an operation commonly called im2col. For example, if the input is $[227\times227\times3]$ and it is to be convolved with $11\times11\times3$ filters at stride 4, then we would take $[11\times11\times3]$ blocks of pixels in the input and stretch each block into a column vector of size $11\times11\times3 = 363$. Iterating this process in the input at stride of $4$ gives $(227-11)/4+1 = 55$ locations along both width and height, leading to an output matrix X_col of im2col of size $[363 x 3025]$, where every column is a stretched out receptive field and there are $55\times55 = 3025$ of them in total. Note that since the receptive fields overlap, every number in the input volume may be duplicated in multiple distinct columns.
2. The weights of the CONV layer are similarly stretched out into rows. For example, if there are $96$ filters of size $[11\times11\times3]$ this would give a matrix W_row of size $[96 \times 363]$.
3. The result of a convolution is now equivalent to performing one large matrix multiply np.dot(W_row, X_col), which evaluates the dot product between every filter and every receptive field location. In our example, the output of this operation would be $[96 \times 3025]$, giving the output of the dot product of each filter at each location.
4. The result must finally be reshaped back to its proper output dimension $[55\times55\times96]$.

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt

def im2col(img, kerSize, stride):
    #######################################################################################
    # A naive implementation of im2col function, if you really want to use it in a model 
    # scikit-image has  a very optimized version of this function known as view_as_windows
    #######################################################################################
    
    # Generating Image Matrix
    image = np.zeros((kerSize[0] * kerSize[1],(img.shape[0]-4)*(img.shape[1]-4))) 
    row = col = 0 
    for iterator in range(image.shape[1]):
        image[:,iterator] = img[row:row+kerSize[0],col:col+kerSize[1]].ravel().transpose()
        #print('row: {} col: {}'.format(row, col))
        if img.shape[1] == col + kerSize[1]:
            row = row + stride
            col = 0
        else:
            col = col + stride
    return image

def calOutShape(imgSize, kernelSize, stride, padding=0):
    #####################################
    # Function to calculate output shape
    #####################################
    return((imgSize[0] - kernelSize[0] + 2*padding)/stride + 1, (imgSize[1] - kernelSize[1] + 2*padding)/stride + 1)
        
if __name__=='__main__':

    # Reading in an image
    img = cv2.imread('/home/kpit/pic.png',0)
    print('Original Image Size: {}'.format(img.shape))
    
    # Generating a 5 X 5 Laplacian kernel
    kernel = np.ones((5,5)) * -1          
    kernel[2,2] = -24
    print('Kernel Size: {}'.format(kernel.shape))
    
    stride = 1              # Setting Stride 
    padding = 2             # Setting Padding
    
    outShape = calOutShape(list(img.shape), list(kernel.shape), stride, padding)  # Calculating Output Shape
    print('Calculated Output shape: {}'.format(outshape))
    
    
    # Creating new image with padding
    newImg = np.zeros((int(outShape[0] + 2*padding), int(outShape[1] + 2*padding)))
    newImg[padding:-padding,padding:-padding] = img
    
    # Generating column image
    colImage = im2col(newImg, kernel.shape, 1)
    print('Column image size: {}'.format(colImage.shape))
    

    # Performing convolution as matrix multiplication
    convImg = kernel.ravel().dot(colImage)
    convImg = convImg.reshape(img.shape)
    convImg = convImg/25
    print('convImg Size: {}'.format(convImg.shape))

    # Displaying output
    plt.subplot(121), plt.imshow(img, cmap='gray')
    plt.subplot(122), plt.imshow(convImg, cmap='gray')
    plt.show()

#### Dilated Convolution
So far we’ve only discussed **contiguous CONV filters**. However, it’s possible to have filters that have spaces between each cell, called dilation. As an example, in one dimension a filter $w$ of size $3$ would compute over input x the following: $w[0] \times x[0] + w[1] \times x[1] + w[2] \times x[2]$. This is dilation of $0$. For dilation 1 the filter would instead compute $w[0] \times x[0] + w[1] \times x[2] + w[2] \times x[4]$; In other words there is a gap of $1$ between the applications. This can be very useful in some settings to use in conjunction with $0$-dilated filters because it allows you to merge spatial information across the inputs much more agressively with fewer layers. For example, if you stack two $3 \times 3$ CONV layers on top of each other then you can convince yourself that the neurons on the 2nd layer are a function of a $5 \times 5$ patch of the input (we would say that the effective receptive field of these neurons is $5 \times 5$). If we use dilated convolutions then this effective receptive field would grow much quicker.

#### Pooling Layer
It is common to periodically insert a Pooling layer in-between successive Conv layers in a ConvNet architecture. Its function is to progressively reduce the spatial size of the representation to reduce the amount of parameters and computation in the network, and hence to also control overfitting. The Pooling Layer operates independently on every depth slice of the input and resizes it spatially, using the MAX operation. The most common form is a pooling layer with filters of size 2x2 applied with a stride of 2 downsamples every depth slice in the input by 2 along both width and height, discarding 75% of the activations. Every MAX operation would in this case be taking a max over 4 numbers (little 2x2 region in some depth slice). The depth dimension remains unchanged. 

![Maxpool Operation](Images/maxpool.jpeg)

# Convolutional Neural Network Application:
## Semantic segmentation
In this particular task, we don't just want to classify the image, we want to output a decision of a category for every pixel in that image. Observe the following examples, lets look at the cat example in particular, for every pixel in that image we want to categorize it as grass, cat, sky, tree etc. We will have a fixed number of category, but unlike earlier when we assigned a single category to an image, now we want to assign one category to each pixel of that image.

One thing to note here is that the output **does not distinguish between multiple instances** of the same object. The example where two cows are standing very close to each other, it looks like they are marked a single object in image.

![Semantic Segmentation](Images/semanticSegmentataion.png)

### Sliding Window
Now one way you can approach this problem is by **extracting multiple pathces** from the image and treat this as a classification problem. Whatever category gets assigned to the image, the same category can be assigned to the central pixel. The following diagram shows the approach-

![Sliding Window Approach](Images/slidingWindow.png)

This model could work but, we must have a croped image for every pixel in the image. As you can probably guess working through forward and backward pass will be computationally very expensive. when we try to classify two patches that are right next to each other, then the overlapping part of the image will go through the same layers of convolution and these computations can be shared. But this idea is not practicle, but it is a very simple approach to this problem.

### Fully Convolutional Network
In this approach we remove the fully connected layers of the classification network and place more convolution layer. In this way we preserve the spatial dimentions of the image and in the end we might get a tensor which has height and width as same as the input image and class scores for every pixel. For example our image is of size $100 \times100\times3$, the output would look something like $100\times100\times20$, where $20$ is the number of classes, we need to categorize each pixel into. We can take a classification loss at every pixel and average them to get a final loss.Now we can train this network with regular backpropagation.

![Fully Convolutional Network](Images/fullyConectedNetwork.png)

As you can see performing multiple **convolution on a high resolution image is computationally expensive**, we generally perform **downsampling** and **upsampling** multiple times in the network. Now the modified architecture look like this-

![Fully Convolutional Network](Images/fullyConvolutionalNetwork2.png)

We have seen downsampling before, but we havent really seen upsampling yet. Lets explore some of the techniques to upsample an image. One of the techniques is called **nearest neighbour unpooling** which copies the data from a region of downsampled image multiple times to the corresponding region in the upsampled image. Another technique is called **"bed of nails"** upsampling which have most of the elements as zeros in the upsampled image and values from the downsampled image in corresponding region of upsampled image. Generally we put these values in the upper left corner.This is illustrated by the image below-

![Upsampling Techniques](Images/upsampling.png)

Most of these fully convolutional networks are highly symmetrical, therefore we can take advantage of these symmetry by connecting the downsampling part of the network to the corresponding upsampling part for better upsampling. In this process we remember the position of element that was maximum in a particular region and while upsampling, then we perform something similar to "bed of nails" upsampling except we now place these values at the position that we remembered from downsampling(and not at the top left corner). This is called **Max-Unpooling**.

![Max-Unpooling](Images/maxUnpooling.png)

The techniques we have seen so far are fixed functions, but we can do something a bit more interesting. We can also make this part learnable by using a technique called **Transposed Convolution**. The process is semantically opposite to that of convolution. In convolution, we take a part of the image and perform inner product with the filter, but here the input will be scalar which we will scale the values of the filter by this value to generate a matrix of values, which will fill the output image. Sometimes a part of output image will overlap between two transposed convolutions, in that case we will sum those values up. But this sum operation that we are doing sometimes generate checker-board kind of artifact in the output. Therefore people generally use $4\times4$(stride-2) or $2\times2$(stride-2) convolution, it helps counter this issue.

![Transposed Convolution](Images/transposedConv.png)

![1-D Transposed Convolution](Images/1dTransConv.png)

### Classification + Localization
Sometimes just knowing, what's in the image is not enough. **Location** of a object in the image plays a huge role in any kind of computer vision application. Therefore in addition to figuring out what's in the image we will also create a bounding box around the object. In this particular task(Classification + Localization) we know ahead of time that it contains an object and we are going to produce exactly one bounding box around that object.

For this particular task we will be using a lot of the machinery that we already built. As you can see from the diagram below, instead of having a final vector providing us a class score, we will connect an additional fully connected layer that will produce four number for the bounding box(x,y,width,height). 

![Network Architecture](Images/classlocal.png)

When training the above network we will have two separate losses one for the classification head(Softmax) and the other for the regression head(L2 loss). We generally take a weighted sum of these two losses and the weight is controlled by a different hyperparameter. This particular hyperparameter is different from the hyperparameters that we have seen previously. Hence  a good strategy will be to take a separate loss and see the performance of different values of this hyperparameter over that loss function. We can also use this to estimate human pose by estimating the position of the joints, because this can also be treated as a regression problem.

![Human Pose Estimation](Images/humanPoseEstimation.png)

![Human Pose Estimation](Images/poseEstimation.png)

## Object Detection
Object detection is different from classification+localization problem as the image may contain multiple instances of different classes. This is the reason why performing regression is not feasible as the number of object to perform regression is not know in advance. This number can vary from image to image.

### Sliding Window
In this method we will plot the slide a window that represent our region of interest in the image and we will try to correctly classify the object in this window simultaneously producing bounding box for the same. We will add an additional category "Background" which will represent anything except the classes we are interested in. But as you can guess, sliding this window over all the patches in the image is computationally expensive. This is the reason that this method is not used in practice.

![Object Detection(Sliding Window)](Images/detectionSliding.png)

### Region Proposal
Well we can always do better than the brute force. The next idea is to use a technique which uses traditional image processing techniques to propose a number of regions in the image where the object might be present. This technique will try to find out blobby regions that are likely to contains object. For example a technique know as **Selective Search** will propose $2000$ region proposals given an input image. Selective search has a pretty high recall.

![Region Proposal](Images/regionProposal.png)

### R-CNN
The idea of region proposals was really put together very nicely by R-CNN. First we run some region proposal technique(ex. Selective Search) on the image to get a set of proposed regions(nearly 2K image patches), then we warp these Images to form a fixed size image for the convolutional neural network. The CNN's will then extract features necessary for the classification, which will used by a SVM to assign category for each proposed region. Now we will also have another SVM head which will predict the correction in the bounding box that was proposed by the region proposal method. Sometimes the predicted bounding box can also be present outside the proposed region and generally a lot of proposed regions will not correspond to any of the objects that we are interested in, so we generally categorize these regions as "Background class".

![R-CNN](Images/R-CNN.png)

#### Problem:
* Training takes a lot of time(84 hrs) and takes a lot of disk space.
* Inference timie is also pretty large(47 sec).

### Fast R-CNN
To overcome some these issues of people came up with Fast R-CNN, which cleverly reduces the inference time by reusing a lot of computation. instead of applying selective search on the entire image, we take our regions from the fifth convolution layer's feature map. Now we can reuse the computation performed in computing the convolution. We will now perform warping operations on these regions to make them compatible with the fully connected layers downstream, which will predict class and position of the boinding box. This network is 10 times faster to train and at inference time the major part of the computational time is for generating region proposals. So the computations are bottlenecked by the region proposal method. 

![Fast R-CNN](Images/fastR-cNN.png)

### Faster R-CNN
Faster R-CNN overcome this computational bottleneck issue by using a **Region Proposal Network**, which will generate proposals. These process of generating region proposals is actually leared by the network during training. After passing the image through convolutional layer, we propagate the generated convolutional feature map to a region proposal network. The region proposal network will now generate some region proposals, which will passed through ROI pooling and then sent to fully connected networks to make the final prediction. But now as you can see the loss functions have increased from two to four. The classification loss at the region proposal network decides whether the region contain an object or not, so any proposal which have very less overlap with a object should be consider as negative.

![Faster R-CNN](Images/fasterR-CNN.png)

![Inference Time Comparision](Images/comparisionInferenceTime.png)

### Detection without proposal
Some of the detection techniques does not rely on region proposals, instead they divide the whole image into some course grid. For each cell in the grid, we assume a set of base bouning boxes of different dimention. Now for each of these base bounding boxes, we will predict classifiaction score and offset to the actual bounding box. Some examples of these type of networks are YOLO( You Only Look Once) and SSD( Single Shot Detection).

![Detection Without Proposals](Images/yolo.png)

The region based family of methods provide high accuracy but tends to be a bit slower than the single shot methods. 

### Instance Segmentation
In instance segmentation problem given an image we want to predict class and location of the object in the image but, rather than just predicting a bounding box for the objects, we also want to know which pixel in the image correspond to which object instance. So this is kind of hybrid between object detection and segmentation. A big part of the network look just like R-CNN, in addition to predicting a clssification score and a boundind box, we project the convolution feature map onto a convolution layer to predict a segmentation mask for each of the region proposed. This mask is then used to perform semantic segmentation inside the predicted bounding box.

![Mask R-CNN](Images/maskR-CNN.png)

## Object Detection Task 
In this section we are going to look at some of the recent advancement's and popular techniques being used for object detection task. 

### A Unified Multi-scale Deep Convolutional Neural Network for Fast Object Detection (MS-CNN)
Classical object detectors, based on the sliding window paradigm, search for objects at multiple scales and aspect ratios. While real-time detectors are available for certain classes of objects, e.g. faces or pedestrians, it has proven difficult to build detectors of multiple object classes under this paradigm.

The more recent approaches like **Faster-RCNN** addresses the issue with a **region proposal network (RPN)**, which enables end-to-end training. However, the **RPN generates proposals of multiple scales by sliding a fixed set of filters over a fixed set of convolutional feature maps**. This creates an inconsistency between the sizes of objects, which are variable, and filter receptive fields, which are fixed. This compromises detection performance, which tends to be particularly poor for small objects, like that in the center of figure below. In fact we can handle such objects by upsampling the input image both at training and testing time. This increases the memory and computation costs of the detector. 

This work proposes a unified multi-scale deep CNN, denoted the multi-scale CNN (MS-CNN), for fast object detection. The network consists of two sub-networks: an object proposal network and an accurate detection network. Both of them are learned end-to-end and share computations. However, to ease the inconsistency between the sizes of objects and receptive fields, object detection is performed with multiple output layers, each focusing on objects within certain scale ranges (see Fig. 3). The intuition is that lower network layers, such as “conv-3,” have smaller receptive fields, better matched to detect small objects. Conversely, higher layers, such as “conv-5,” are best suited for the detection of large objects. The complimentary detectors at different output layers are combined to form a strong multi-scale detector. A second contribution of this work is the use of feature upsampling as an
alternative to input upsampling. This is achieved by introducing a deconvolutional layer that increases the resolution of feature maps, enabling small objects to produce larger regions of strong response. Without image upsampling, the MS-CNN achieves speeds of 10 fps on KITTI (1250$\times$375) and 15 fps on Caltech (640$\times$480) Images.

![MS-CNN Architecture](Images/MS_CNN.png)

The network has a standard CNN trunk, depicted in the center of the figure, and a set of output branches, which emanate from different layers of the trunk. These branches consist of a single detection layer. Note that a buffer convolutional layer is introduced on the branch that emanates after layer “conv4-3”. Since this branch is close to the lower layers of the trunk network, it affects their gradients more than the other detection branches. This can lead to some instability during learning. The buffer convolution prevents the gradients of the detection branch
from being back-propagated directly to the trunk layers. This approcah is shown to produce accurate object proposals on detection benchmarks with large variation of scale, such as KITTI, achieving a recall of over 95% for only 100 proposals. 

![Dtection part of the network](Images/detectNet.png)

The network detects objects through several detection branches. The results by all detection branches are simply declared as the final proposal detections. The network has a standard CNN trunk, depicted in the center of the figure, and a set of output branches, which emanate from different layers of the trunk. These branches consist of a single detection layer. Note that a **buffer convolutional layer** is introduced on the branch that emanates after layer “conv4-3”. Since this branch is close to the lower layers of the trunk network, it affects their gradients
more than the other detection branches. This can lead to some instability during learning. **The buffer convolution prevents the gradients of the detection branch from being back-propagated directly to the trunk layers**.

#### Feature Map Approximation
Input size has a critical role in CNN-based object detection accuracy. Simply forwarding object patches, at the original scale, through the CNN impairs performance (especially for small ones), since the pre-trained CNN models have a natural scale (e.g. 224×224). While the R-CNN naturally solves this problem through warping, it is not explicitly addressed by the Fast-RCNN or Faster-RCNN. To bridge the scale gap, these methods simply upsample input Images (by ∼2 times). For datasets, such as KITTI, containing large amounts of small objects, this has limited effectiveness. Input upsampling also has three **side effects**: 
* large memory requirements
* slow training
* slow testing. 

It should be noted that input upsampling does not enrich the image details.A Unified Multi-scale Deep CNN for Fast Object Detection. Instead, it is needed because the higher convolutional layers respond very weakly to small objects. For example, a 32×32 object is mapped into a 4×4 patch of the “conv4-3” layer and a 2×2 patch of the “conv5-3” layer. This provides limited information for 7×7 ROI pooling. To address this problem, we consider an efficient way to increase the resolution of feature maps. This consists of upsampling feature maps (instead of the
input) using a deconvolution layer, as shown in figure above. **Here input rescaling is replaced by feature rescaling**. A feature approximator is learned by least squares. In the CNN world, a better solution is to use a deconvolution layer. Unlike input upsampling, feature upsampling does not incur in extra costs for memory
and computation. Our experiments show that the addition of a deconvolution layer significantly boosts detection performance, especially for small objects. To the best of our knowledge, this is the first application of deconvolution to jointly improve the speed and accuracy of an object detector.

### Spatial Pyramid Pooling
The prevalent CNNs require a fixed input image size (e.g., 224×224), which limits both the aspect ratio and the scale of the input image. When applied to Images of arbitrary sizes, current methods mostly fit the input image to the fixed size, either via cropping or via warping, as shown in Figure below. **But the cropped region may not contain the entire object, while the warped content may result in unwanted geometric distortion**. On the other
hand, the fully-connected layers need to have fixed size/length input by their definition. Hence, the fixed size constraint comes only from the fully-connected layers, which exist at a deeper stage of the network.

![Issues with Warping and Croping](Images/warp.png)

To adopt the deep network for Images of arbitrary sizes, we replace the last pooling layer (e.g.,pool 5 , after the last convolutional layer) with a spatial pyramid pooling layer. Figure below illustrates the method. In each spatial bin, we pool the responses of each filter (throughout this paper we use max pooling). The outputs of the spatial pyramid pooling are **kM - dimensional vectors with the number of bins denoted as M** (k is the number of filters in the last convolutional layer). The fixed-dimensional vectors are the input to the fully-connected layer.

![Spatial Pyramid Pooling](Images/spatialPyramid.png)

The forward and backward pass of Spatial Pyramid Pooling:

| Forward Pass | Backward Pass |
| :---: | :---: |
| ![Spatial Pyramid Pooling Backward Pass](Images/sppooling.png) | ![Spatial Pyramid Pooling Forward Pass](Images/sppBack.png) |

## DenseBox: Unifying Landmark Localization with End to End Object Detection
DenseBox is a unified end-to-end FCN framework that directly predicts bounding boxes and object class confidences through all locations and scales of an image. DenseBox does not require proposal generation and is able to be optimized end-to-end during training. Although similar, DenseBox is more carefully designed to detect objects under small scales and heavy occlusion. Training DenseBox involves applying careful hard negative mining techniques to boostrap the detection performance.

![Basic Dense Box Architecture](Images/denseBox.png)

### Ground Truth Generation
It is unnecessary to put the whole image into the network for training because it would take most computational time in convolving on background. A wise strategy is to crop large patches containing faces and sufficient background information for training.

In training, the patches are cropped and resized to 240 × 240 with a face in the center roughly has the height of
50 pixels. **The output ground truth in training is a 5-channel map sized 60 × 60 , with the down-sampling factor of 4**. The positive labeled region in the first channel of ground truth map is a filled circle with radius $r_c$, located in the center of a face bounding box. **The radius $r_c$ is proportional to the bounding box size, and its scaling factor is set to be 0.3 to the box size in output coordinate space**, as show in figure below. The remaining 4 channels are filled with the distance between the pixel location of output map between the left top and right bottom corners of the nearest bounding box. 

![Ground Truth Generation](Images/gdtruth.png)

#### Output Probability Map

![Detecting Object a multiple scale](Images/probOutput.png)

### Network Architecture
The network architecture illustrated in figure below, is derived from the VGG 19 model used for image classification. The whole network has 16 convolution layers, with the first 12 convolution layers initialized by VGG 19 model. The output of conv4 4 is feed into four 1 × 1 convolution layers, where the first two convolution layers output 1-channel map for class score, and the second two predict the relative position of bounding box by 4-channel map. The last 1 × 1 convolution layers act as fully connected layers in a sliding-window fashion.

![Dense Box Architecture](Images/archDenseBox.png)

### Multi-Task Training Loss
Like Fast R-CNN, the network has two sibling output branches. The first outputs the confidence score $ŷ$ (per pixel in the output map) of being a target object. Given the ground truth label $y$ $*∈\{ 0, 1 \}$ , the classification loss can be defined as follows.

$$ L_{cls}(ŷ, y^∗ ) = \|{ŷ − y^∗}\|$$

The second branch of outputs the bounding-box regression loss, denoted as $L_{loc}$ . It targets on minimizing the $L2$ loss between the predicted location offsets $\hat{d} = ( \hat{d}_{tx} , \hat{d}_{ty} , \hat{d}_{tx} , \hat{d}_{ty} )$ and the targets $d^∗ = (d^∗_{tx} , d^∗_{ty} , d^∗_{tx} , d^∗_{ty} )$, as formulized by:

$$ L_{loc}(\hat{d},d^*) = \sum_{i∈\{tx,ty,bx,by\}} {\|\hat{d_i} - d^*_i\|}^2$$



## Vehicle Detection and Pose Estimation
The vast majority of existing object detectors focuses on finding 2D bounding boxes , which provide sufficient information for basic reasoning about object positions. However, it is insufficient for autonomous driving
applications, where finding poses of objects in the 3D world is desired. The topic of this work is detection of cars and representing their pose with 3D bounding boxes. The 3D bounding box is a very convenient and, for many
applications, sufficient representation of the objects in the 3D world. We, in line with general practice, define the 3D bounding box as a tight rectangular cuboid around an object that has 9 degrees of freedom (3 for position, 3 for rotation, and 3 for dimensions). This information is sufficient to determine the position, orientation, and size of the object in the 3D world, which can be used especially for path planning of an autonomous car.

![2D and 3D Bounding Box](Images/3dBox.png)

The work has selected DenseBox as the base model because of the following features. DenseBox can be easily adapted to predict the image projections of the corners of 3D bounding boxes and its loss function is way simpler to compute. Which allows everyone to easily train, test their progress, and tune parameters on a single scale detector, which we then extend to a multi-scale one-pass detector.

#### Input Layer
The network takes 3-channel RGB Images on the input. For detection, the Images can be of arbitrary resolution but the intensities of the image pixels $[0, 255]$ are converted to span the interval $[-1, 1]$ in the following way:
$$ \frac{v-128}{128} $$

#### Bounding Box Sampling
The detector has to learn to detect cars (bounding boxes) of various sizes. The original sizes in the training set would not be enough, and hence we perform cropping of bounding boxes to random scales.

![Bounding Box Sampling](Images/BoundingBox.png)

#### Output Layers
The output layer have either **5 (for 2D bounding box)** or **8 (for 3D bounding box)** channels, whose dimensions are down-sampled with respect to the input image by the scale factor s. In DenseBox, the factor s = 4 is used, however, in our case we are performing multi-scale detection directly in one pass through the network, hence we have several scales s on the output, usually 2, 4, 8 or 2, 4, 8, 16, depending on what sizes of objects we aim to detect.

![Output Layer Layout](Images/outLayer.png)

| | | | |
| :---| | | :--- |
| **prob**: Probability of being a Vehicle | | | **prob**: Probability of being a Vehicle |
| **xmin**: x-coordinate of top left corner of the bounding box | | | **fblx**: front bottom left x-coordinate |
| **ymin**: y-coordinate of top left corner of the bounding box | | | **fbly**: front bottom left y-coordinate |
| **xmax**: x-coordinate of bottom right corner of the bounding box | | | **fbrx**: front bottom right x-coordinate |
| **ymax**: x-coordinate of bottom right corner of the bounding box | | | **fbry**: front bottom right y-coordinate |
| | | | **rblx**: rear bottom left x-coordinate |
| | | | **rbly**: rear bottom left y-coordinate |
| | | | **ftly**: front top left y-coordinate |


#### Target Representation
The pixels in the response map within some given radius r = d/2 from an object center (center of its 2D bounding box) are responsible for detecting that object and regressing the coordinates of its 2D or 3D bounding box . In the case of a 2D bounding box we regress the coordinates of the top-left and bottom-right corners of the 2D bounding box. In the case of a 3D bounding box we regress the position of the projections of the rear-bottom-left, front-bottom-left, and front-bottom-right corners and the y-coordinate of the front-top-left corner.

![Target Representation](Images/targetRep.png)

**Gaussian response:** In our target probability response map we require either 0 for the pixels out of the object centers or 1 for the pixels in the object centers. This results in a steep change in the error on the edge between a correct and incorrect detection when the network output is slightly misplaced. Using a Gaussian instead guides
the learning algorithm better towards the required position. Therefore, we filter the ground truth probability response map with a 3 $\times$ 3 Gaussian filter with $\sigma$ = 1. This results in having Gaussian blobs in the positions of the circles and smoother transitions (smaller error) for small misplacements in the response maps.

![Gaussian Error](Images/gaussianError.png)

| **Left**:The response of a network trained on a binary response map | **Right**:Its Error when compared to ground truth |
| :--- | :--- |
| | |


#### Corrdinate Response Map
Coordinate response maps So far, we have been describing the probability response map channel, however, the output contains other 4 (or 7) channels that regress the 2D (or 3D) bounding box coordinates. The coordinates are necessarily relative to the coordinate map pixel position in which they are regressed as we are using a FCN. They are regressed in each pixel (understand pixel as a sample with 5 or 8 channels), which has a value > 0 in the probability channel. This is because each positive pixel (sample) in the net output represents a bounding box.
The relative coordinate values are scaled to approximately (0, 1), which is more suitable for training of the network - provides gradients of similar magnitude to the ones from the probability channel. The relative coordinate value v is converted to the value $v^{'}$ in a coordinate response map in the scale s i as follows:

$$ v^{'} = \frac{v}{size_i} + 0.5 $$

where $size_i$ is the ideal object size for the scale $s_i$ . This assures that a 2D bounding box with the dimensions $size_i \times size_i$ will have coordinates in the relative coordinate maps $x^{'}_{min} = 0$, $y^{'}_{min} = 0$, $x^{'}_{max} = 1$, and $y^{'}_{max} = 1$. Other bounding box dimensions will have values around 0 and 1.

#### Loss Function
The loss function is based on the widely used squared Euclidean loss function:

$$ E = \frac{1}{2}\sum^{N}_{i=1}{(t_i - y_i)}^2$$

where $t_i$ is the target value (ground truth) and $y_i$ is the network output, i iterates over all N output layer neurons.

#### Detection Extraction
When an image passes through our detection network, it produces a multi-channel response map (or a set of multi-channel response maps in the case of multi-scale detection). Each pixel in such a response map represents one detected bounding box (2D or 3D) and the probability of the bounding box being an object. 

#### Ground Truth Association
In both cases of 2D and 3D bounding box, we require all objects (cars), which the annotator can distinguish in the
image to be annotated. **The annotator should label cars of all sizes, occluded up to 90%, and truncated up to 75%**. For the annotators this essentially means to label all objects, which they can recognize from the single image without having any extra knowledge about the scene.

![Ground Truth Assocation](Images/annotation.png)

**2D bounding boxes** are minimum enclosing image-axis-aligned rectangles, which are required to encompass the whole object - in our case the car. However, they are only useful if we can reason from them about the object size or even pose. This leads to the labeler having to make up the hidden (occluded or truncated) parts of the objects.

![2D Bounding Box Annotation](Images/anno2D.png)

**3D bounding boxes** are minimum enclosing rectangular cuboids of objects in the real 3D world. In the case of a 3D bounding box, occlusion and truncation translates to 3D as missing parts (missing parts of a point cloud created from a stereo image pair) of the objects. Again, the labeler needs to think up these missing parts and draw a
tight rectangular cuboid encompassing the whole object in the 3D world. On top of that, each 3D bounding box has a distinguished front and rear side, which determine the pose of the object.

![3D Bounding Box Annotation](Images/3DBoudingBox.png)

#### Representation
In order to work with the 2D and 3D bounding boxes autothors had to select the most suitable representation for each and so they created their own annotation file formats, which keep the annotations in the specified forms.

#### 2D Bounding Box - BBTXT
The 2D bounding boxes are represented by the coordinates of their top-left $(x_{min} , y_{min} )$ and bottom-right $( x_{max} , y_{max} )$ corners.

![2D Bounding Box Representation](Images/2DRep.png)

#### 3D Bounding Box - BB3TXT
For 3D bounding boxes we will store the coordinates of the projections of the bounding box corners into the image. This representation is very convenient as it makes the coordinates independent of the used camera (and its image projection matrix). We can make use of it in our network output as it allows us to train a single network
and then use it on Images taken by any other camera (for example a different dataset).

![3D Bounding Box Representation](Images/3DBox.png)

![3D Bounding Box Representation](Images/3DRep.png)

#### Groud Plane Extraction
The aforementioned 3D bounding box representation requires the ground plane equation to be known in order to be able to reconstruct the bounding boxes in the 3D world. Therefore, for each image, we need the equation of the ground plane. For each 3D bounding box there are 4 vertices, which lie in the ground plane. This gives us 158388 vertices if we accumulate the bounding boxes from the whole training set. We use **RANSAC to find the ground plane, which suits those vertices the best**.

Given the number of iterations N and a set of vertices G in the ground plane we can run the RANSAC estimation. A plane is defined by 3 vertices $p_1$ ,$p_2$ , $p_3$ and can be computed as follows:

$$l_1 = p_2 - p_1 $$
$$l_2 = p_3 - p_1 $$
$$n = l_1 \times l_2 $$
$$d = -(n · p 1 )$$

**PGP Files**: In order to be able to use this information for 3D bounding box reconstruction, we created a new file format called PGP, which contains the $3 \times 4$ image projection matrix $P$ and the coefficients of the ground plane equation $ax + by + cz + d = 0$ for each image. The file looks as follows:

![PGP Files Format](Images/pgp.png)




#### Performance Measure
lets look at some performance measures, which are used throughout the whole evaluation process-
**Intersection over union (IOU)**: It measures the similarity of two bounding boxes. It is used to compare detected bounding boxes and ground truth bounding boxes.
$$ IOU(b_1,b_2) = \frac{A(b_1 \cap b_2)}{A(b_1 \cup b_2)}$$

where $A(·)$ represents area and b are bounding boxes. For our evaluation we require $IOU \geq 0.5$ to accept a detection as a correct one, in the KITTI benchmark they use $IOU \geq$ 0.7$ for the car detection task.

**Precision**: It is the ratio of the detected objects $TP + FP$ , which are detected correctly. It tells us how many false positive $FP$ detections the detector produces. It is defined as follows-
$$ Precision = \frac{TP}{TP+FP}$$

**Recall**: It tells us the ratio of the ground truth objects $TP + FN$ , which are detected by the detector. It is defined as-
$$ Recall = \frac{TP}{TP+FN}$$

**Average precision (AP)**: is used in the KITTI benchmark. It samples the precision/recall curve on 11 places, interpolates it and computes the arithmetic mean. Therefore, this measure gives insight into how the precision/recall curve changes.

**Mean distance error (MDE)**: It gives an insight into the performance of a 3D bounding box detector. The detections are matched to the ground truth using IOU of 2D bounding boxes. After that, the distance of the detection - ground truth pairs is computed in the 3D world. An MDE plot is created by splitting the pairs into
groups by the distance from the camera. This provides an insight on how the error in position changes with respect to the distance of the bounding boxes from the camera.

#### Learning Rate
The learning rate schedule was modified  as follows. Starting with $l_r = 0.0001$ and then raising the learning rate to $l_r = 0.0005$ after 500 iterations. For training the multi-scale network r2_x2_to_x16_s2 we had to change this schedule even more as the gradient was unstable for a longer period of time and the used learning rate was on the edge between successful training and divergence. Since we were also using more training iterations (100000) we raised the learning rate once $2.5\times$ at iteration 10000 and again $2.5\times$ at iteration $20000$. This resulted in even higher learning rate in the end, but it was used in the stable part of the training, which was
beneficial.

As we are using FCN, the gradient of the loss function with respect to each weight depends on the number of the pixels of the output response map because the convolutional operators introduce weight sharing. The aforementioned networks are trained on $256 \times 128$ Images, which means that the output response map has $64 \times 32$ pixels. If we train a network on a larger image, say $256 \times 256$ (response map $64 \times 64$), we need to scale down the learning rate accordingly. In this example case we would have to use $l_r = 0.0001/2 = 0.00005$ to achieve the same training behavior.

#### Analysis
The evaluation showed that we designed a new promising method based on the combination of ideas from the DenseBox, SSD, and MS-CNN algorithms, which is capable of detecting 3D bounding boxes of cars from monocular Images. However, as all methods, it has several problems. We now asses these problems and suggest how to tackle them.

**Truncated Cars:** A particular example of where our method lacks behind its competitors is the detection of very truncated cars. Examples of such annotated cars are shown in Fig. below in the bottom-right and bottom-left corner. This limitation comes from the chosen representation on the output. In the current setup, our method requires the
center of the 2D bounding box of a car to be inside of the image in order to be able to detect it.

![Truncated Cars](Images/trunCar.png)

We argue that in practice detection of such cars from a camera is mostly not necessary. Such cars are very close to the camera, therefore, easily detectable by a different kind of sensor. If one would require detection of very truncated cars anyway, the input image can be padded with a black border, and thus provide room for the responses in the centers of the truncated cars.

**Extensively Overlapping Cars:** The detection of cars with large overlap is a problem in general, which the presented detector shares as well. In our case, the problem is that the circles drawn in the center of the 2D bounding boxes of the cars are overlapping if the cars are in the same scale (response map) and occlude each other a lot. 

![Overlapping Cars](Images/occluCar.png)

It is difficult even for a human eye to recognize the 6 different bounding boxes in the middle of Fig. above. The problem of our representation is that the circles of very overlapping cars are merged together into a blur. It is even a bigger problem for the bounding box coordinate response maps because there we have to decide which bounding box we are regressing in each pixel. Since, in this case, one pixel in the response map corresponds to multiple bounding boxes, it is a problem.

**3D Pose Estimation:** Another problem is the regression of the coordinates of the projected 3D bounding boxes. We directly predict the coordinates with respect to the response map pixel position regardless of the car orientation. However, our observations suggest that the network has a problem distinguishing the orientation of side-view cars.
As can bee seen in Fig. below, instead of learning the correct coordinates of the projections the network learns to average the coordinates of the left and right facing cars. This results in a completely incorrect car pose, either facing the camera or the other way around, or predicting some small bounding box in the middle.

![3D Pose Estimation](Images/poseEst.png)

The solution to this problem may be to use sub-categories divided by the orientation of the car with respect to the camera. We want to avoid the network to learn to average the coordinates of the left and right facing cars. Creating for example 4 orientation sub-categories (frontal, rear, left, and right view) would prevent this from happening as the coordinates would only be regressed in some given region.

**Double Detection**: Associated with the previous problem is that the detector sometimes tends to predict multiple bounding boxes for a single car. Fig. below shows a case when a detector predicted a side-view bounding box and a rear-view bounding box on the same car. The fact that this is happening makes sense since the two bounding boxes
have centers in different places and, therefore, do not interfere with each other in the probability response maps.

![Double Detection](Images/doubleDetect.png)

It is a problem since the network does not learn to completely suppress the irrelevant view by itself. A thorough solution requires a complex analysis of the detected bounding boxes. One such approach could be non-maxima suppression (NMS) in the 3D world. The two predicted bounding boxes significantly overlap in 3D, and thus filtering them in 3D could bring the suppression of the incorrect view.

**Hard Negatives:** It is a well known problem that neural networks are easily fooled, as described for example in [36]. Very often, they produce detections with high confidence that are in fact incorrect (false positives). Examples of such detections from our 2D bounding box detector are in Fig. below. Without a very thorough examination of the learned network those false detections are inexplicable.

![False Positive Examples](Images/hardNeg.png)

These detections may be caused for example by over-fitting to outliers in the training set, which we can reduce using dropout layers [50]. We did not use this technique because it prolongs training and training our network already took a significant amount of time (5 days for 2D and 10 days for 3D bounding boxes on NVIDIA TITAN X GPU), however it might be a possible upgrade, which would reduce the false positive rate. Another way to reduce the amount of false positives is hard negative mining during training. Such approach is used in the original DenseBox, however since we changed to positive/negative pixel weighting we did not re-implement it here. Hard negative
mining is, however, a known working technique for reducing false positives.

**Ground Plane:** The reconstructed 3D world position of the detected bounding boxes is heavily dependent on the position of the ground plane. The problem is that the requirement of a flat ground plane does not hold for arbitrary scenes. Even in KITTI, where the ground plane is very flat, we can sometimes see huge reconstruction errors . Arguably, this is the biggest drawback of our method. A better solution would be to obtain a more precise ground plane equation from other sensors in the car or use stereo Images.

![Reconstruction Error](Images/reconErr.png)

**3D Bounding Box Non-maxima Suppression:** Currently, we are performing non-maxima suppression of 3D bounding boxes on their 2D projection into the image (the same way as we do it for 2D bounding boxes). This approach is not ideal because it can filter out cars with IOU of their 2D bounding boxes even though they are actually correct detections. Providing we have more precise 3D reconstruction, the 3D bounding boxes could be filtered directly in the 3D world. The requirement of low or none IOU makes perfect sense when used for 3D bounding boxes as no two cars can intersect each other.

**Weight Initialization:** At the current training setup we always initialize the weights randomly from scratch using the Glorot uniform initialization [22]. However, it was shown that using pre-trained weights for initializing the neural network will most likely lead to a superior performance. It would be interesting to try to initialize the weights of the network for example with the VGG [48] pre-trained weights.

**Unsure Ground Truth Coordinates:** In our ground truth specification we require the labeler to make up the part of the bounding box from the occluded or truncated part of the car. It is important since we need to extract the centers of the object during training. However, we are teaching the network to precisely predict the coordinates of the bounding box corners, which in this case may come from imprecisely labeled coordinates since the labeler is just imagining where the car has its boundaries. If the unsure coordinates were marked in the training labels as in Fig. below we could make use of this information during training. The gradient from the unsure coordinates would
simply be nullified, therefore not taking part in training of the network. This way, the network would learn only on the precise labels.

![Unsure Ground Truth Coordinates](Images/unsureGd.png)

